In [1]:
from koselleck import *

In [24]:
def classify_vector_change(row,perc_threshold=75):
    name=row.vector
    ns=str(name).split('.')[0].split('-')
    n1,n2=ns[0],ns[-1]
    res=f'{n1}~{n2}'
    if row.mean_diff_p<0.05 and row.perc>perc_threshold:
        res=f'+{n1}' if row.mean_diff>0 else (f'+{n2}' if n2!=n1 else f'-{n1}')
#         new=(row.mean2>1 and row.mean1<1) if row.mean_diff>0 else (row.mean2<-1 and row.mean1>-1)
        new=(row.mean2>1 and row.mean1<0.75) if row.mean_diff>0 else (row.mean2<-1 and row.mean1>-0.75)
        if new: res=res+'+'
    return res

In [32]:
df=get_dfchange_from_decade_data(wide=False).reset_index()
df['perc']=df.mean_diff_t_abs.apply(lambda x: percentileofscore(df.mean_diff_t_abs, x))
# df

dfchsz=get_dfchange_from_sattelzeit_models().reset_index()
remove_words = set(df.word) - set(dfchsz.word)
df=df[~df.word.isin(remove_words)]
# remove_words=set()
df['class_vec_change']=df.progress_apply(classify_vector_change,axis=1)
dfchsz['class_vec_change']=dfchsz['class_change']
df=df.append(dfchsz)
df.to_csv('data/data.word_changes_classed.csv')
df

100%|██████████| 80420/80420 [00:01<00:00, 41068.57it/s]


,vector,word,mean1,mean2,mean_diff,mean_diff_t,mean_diff_p,mean_diff_t_abs,len1,len2,...,score_diff_abstractness,is_clean_noiseaware,neighborhood1_local,neighborhood2_local,score_diff_t_abstractness,score_diff_p_abstractness,class_abs,class_change,class_signif,class
0,Sing-Plural.M,guile,-0.347493,-0.422681,-0.075187,inf,0.000000e+00,inf,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sing-Plural.M,patronage,1.165569,2.996845,1.831276,-3.669833e+16,0.000000e+00,3.669833e+16,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sing-Plural.M,haft,1.611224,-0.701572,-2.312796,9.222264e+02,0.000000e+00,9.222264e+02,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sing-Plural.M,intellect,-1.374228,0.822721,2.196948,-6.828419e+02,0.000000e+00,6.828419e+02,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sing-Plural.M,fob,0.306604,1.452882,1.146278,-5.629519e+02,6.632584e-319,5.629519e+02,100.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4819,NaN,love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.002294,1.0,"lover (10), passion (10), pity (10), rapture (...","affection (10), passion (10), tenderness (10),...",0.159039,0.875409,Abs~Conc,-Changed,False,Abs~Conc -Changed
4820,NaN,grief,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.016623,1.0,"sorrow (10), anguish (10), sigh (10), woe (10)...","sorrow (10), anguish (10), remorse (10), sadne...",-0.830267,0.417264,Abs~Conc,-Changed,False,Abs~Conc -Changed
4821,NaN,truth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.008670,1.0,"falsehood (10), sincerity (10), fallacy (10), ...","revelation (10), falsehood (10), sincerity (10...",-0.718520,0.481660,Abs~Conc,-Changed,False,Abs~Conc -Changed
4822,NaN,translation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.043636,1.0,"version (10), translator (10), fragment (10), ...","version (10), translator (10), text (10), lexi...",-2.287025,0.034522,Abs~Conc,-Changed,False,Abs~Conc -Changed


In [26]:
df[df.word=='culture'].class_vec_change

1012              +Virtue
1150                 +Pos
2531              +Interp
4389              +Strong
6262               +Human
7940                +Acad
12539               +Abs+
13747           +Pleasure
31351           Woman~Man
31739      Active~Passive
36608          Time~Space
41692    Collective~Indiv
41979           Freq~Freq
42092         Ambig~Ambig
61960          Qual~Quant
85589           Judg~Perc
440              +Changed
Name: class_vec_change, dtype: object

In [27]:
def format_words(wdf,joinby=', ',lim=None,randomize=False):
    wdf=wdf.groupby('word').mean().reset_index()
#     wdf=wdf.sort_values('word')
    wdf=wdf.sort_values('perc',ascending=False)
    words=wdf.drop_duplicates('word').word
    if randomize: words=words.sample(frac=1)
    o=joinby.join([str(x) for x in words.iloc[:lim]])
    if lim and len(words)>lim: o+=f' ...'
    if len(words): o+=f' ({len(words)})'
    return o

In [28]:

def get_vec_change_table(df,add_solo_col=True, remove_noisy=True, remove_words=set(),lim=None):
    if remove_noisy:
        dfchsz = get_dfchange_from_sattelzeit_models()
        remove_words|=set(dfchsz.query('is_clean_noiseaware!=True').index)
    
    ld=[]
    cs=[c for c in df.class_vec_change.unique() if '~' not in c]
    for c1 in tqdm(cs):
        c1w=df[df.class_vec_change==c1].word
        added=False
        for c2 in cs:
            if c2==c1: continue
            c2w=df[df.class_vec_change==c2].word        
            sw=(set(c1w) & set(c2w))
            sw-=remove_words
            cw=df[df.class_vec_change.isin({c1,c2}) & df.word.isin(sw)]
            ostr=format_words(cw,lim=lim)
            ld.append({'words':ostr,'change1':c1,'change2':c2,'size':len(sw)})
            added=True
        if add_solo_col and not added: ld.append({'words':', '.join(c1w),'change1':c1,'change2':'*','size':len(c1w)})
    odf=pd.DataFrame(ld)
    return odf.pivot('change1','change2','words').fillna('')
    # odf        

In [29]:
# words_ever_abs = set(df.query('vector=="Abs-Conc.Median" & (mean1>1 | mean2>1)').word)
# words_ever_abs

In [30]:
odf=get_vec_change_table(
    df,#[df.word.isin(words_ever_abs)],
    remove_noisy=False,
    lim=200,
#     remove_words=remove_words
)
# odf

100%|██████████| 69/69 [00:49<00:00,  1.40it/s]


In [31]:
printm(
    odf[
        ['+Abs+','+Abs','+Conc','+Conc+','+Changed','-Changed']
    ].loc[
        [c for c in odf.index]# if c.startswith('++')]# if c.endswith('+')]
    ].to_markdown()
)

| change1      | +Abs+                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | +Abs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | +Conc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | +Conc+                                                                                                                                                                                                                                                                            | +Changed                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | -Changed                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|:-------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| +Abs         |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                   | feud, childe, longing, lecturer, clue, caring, germ, serf, herald, mania, thinker, semi, chef, negro, rote, ache, convulsion, papacy, goldsmith, seer, sig, rite, anticipation, discouragement, dwelling, role, mood, renunciation, presentment, gurney, derby, aid, graduate, commune, forte, cheque, sob, debility, trash, meantime, comp, thrill, closing, enigma, labor, duo, budget, reprobation, remonstrance, wail, communication, bey, turkey, sine, blockade, guild, array, abstinence, breadth, urge, moralist, note, star, banking, choosing, sharpness, peninsula, humiliation, harp, data, forum, haven, recitative, gateway, demolition, partnership, collector, engineer, dissent, labourer, embryo, lodger, retribution, victor, jubilee, ego, lane, workhouse, basil, ending, parentage, arbitration, ringing, gilt, builder, din, volunteer, mourning, vent, intervention, marquess, moor, street, scaffold, vita, obstruction, chronicle, relaxation, beau, monday, reckoning, opus, hunting, divorce, accounting, saturday, tavern, tuesday, wednesday, entry, jail, vein, promoter, gazette, thursday, turnpike, quid (127)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | retaliation, goo, blight, lip, reservation, indigestion, liar, cycle, democracy, valentine, liver, adjournment, lamb, cross, tinge, tariff, bishopric, axiom, sound, infusion, freeman, unwillingness, sunday, shoemaker, fox, elder, ado, refutation, exaltation, lace, acquittal, park, echo, gentlewoman, celibacy, liter, prologue, intuition, perversion, lint, tenant, deafness, banker, stability, rider, auto, exclusion, prediction, remission, panic, dollar, episode, comrade, cleanliness, bearer, evasion, dit, gall, bloodshed, commissioner, chairman, implication, warren, baker, explosion, meddling, disaffection, vestry, gaol, allowance, inexperience, valuation, chaplain, concession, bombardment, paradox, complication, consistency, butler, inferiority, distrust, narrowness, faithfulness, willingness, rudeness, creed, denial, acquiescence, prohibition, inquest, biographer, ebb, burst, contentment, interpreting, ire, punch, coroner, recorder, ling, indictment, condemnation, diocese, chill, universality, argyle, flash, focus, revolt, disapprobation, coming, tide, impertinence, unhappiness, viscount, hardship, explanation, advocate, scepticism, cathedral, square, fanaticism, coquette, blindness, assent, melancholy, nerve, comprehension, barrister, niece, believer, apoplexy, mademoiselle, friday, misrepresentation, bid, widower, surveyor, app, limitation, walker, brandy, rector, vocabulary, snow, hierarchy, bacon, pang, intoxication, ban, dean, blasphemy, canto, theory, painter, gladness, priesthood, bologna, antipathy, tea, relict, epidemic, region, phraseology, nightingale, impeachment, misconduct, frankness, infirmary, bitterness, afternoon, vault, inconsistency, vicar, reliance, kinsman, eon, fertility, broker, hurricane, deanery, cam, plaintiff, brevity, banishment, corps, penny, storm, calmness, countess, oil, brick, depth, gentile, chi, executioner, deacon, apothecary, quarter, slaughter ... (295)                                                                                                                                                                                                                              |
| +Abs+        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                   | regime, drawback, canning, loathing, annoyance, retention, congratulation, veto, advert, trait, advent, hostility, goodwill, supporter, lunacy, endowment, delineation, viewing, threat, incorporation, cessation, consummation, confiscation, ruler, desertion, privation, boon, semblance, lending, deduction, lunatic, sect, irritability, craving, auditor, imbecility, symptom, scrutiny, licensing, ballot, detention, disaster, compulsion, culture, variance, exhaust, absence, lifetime, ascendant, accord, import, seizure, grace, mover, sneer, agitation, par, suppression, transfer, piracy, taint, formula, manifesto, delegate, irony, patient, theme, destiny, peril, fellowship, chagrin, intrigue, correction, currency, longevity, hearing, solemnity, warning, heroine, abbe, strife, arc (82)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | deception, regeneration, exaggeration, appropriation, martyrdom, layman, stagnation, arrest, checking, hearer, distraction, enthusiast, rebuke, mediocrity, inaccuracy, follower, reformer, irritation, churchman, sadness, accomplice, recoil, carelessness, extinction, sphere, fervour, proclamation, judaism, bane, learner, multiplication, sacrilege, default, penalty, courtesy, abatement, delirium, verdict, coldness, prosecutor, consecration, value, mortification, critique, formality, license, magnitude, menace, relapse, impetuosity, art, interpreter, courtship, archdeacon, champion, modification, visitation, rumour, absolution, fluency, bankruptcy, hindrance, massacre, blunder, impediment, philology, lord, ordination, disorder, licence, canon, stress, comment, disciple, command, calculation, weariness, exposition, confessor, reluctance, outcry, emotion, interruption, trial, synod, offender, contagion, chancellor, purgatory, wretchedness, insurrection, archbishop, martyr (93)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Acad        | drawback, absence, lending, philology, culture, career, amount, variance, command, learner, grace, hearing, arc, habit, warrant, stress, solemnity (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | communication, sunday, thinker, earnestness, aid, infirmary, graduate, engineer, goal, monday, friday, street, collector, afternoon, saturday, intuition, rapidity, acuteness, interpreting, council, surveyor, builder, chapel, clerk, vocabulary, expiration, cheque (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | majesty, preservation, kingdom, accommodation, hall, article, family, person, interest, study, promise, zeal, succession, authority, advantage, design, pleasure, care, example, manufacturer, assistance, foot, support, goodness, foundation, concert, proof, carpet, concern, treasure, honour, power, relief, manufacture, posterity, exertion, resolution, perseverance, credit, commerce, chemistry, favour, esteem, prejudice, preserving, name (46)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | loon, jelly, room, rill, cottage (5)                                                                                                                                                                                                                                              | graduate, thinker, aid, drawback, rill, pretension, lending, collector, culture, preservation, cheque, deficiency, elocution, furnishing, absence, communication, manual, rendering, cheapness, engineer, loading, repute, dispatch, teacher, correspondence, room, grace, equipment, variance, handling, enlargement, advance, treasure, detail, posture, certificate, cottage, demand, builder, report, access, condition, attendance, vacation, repair, solemnity, service, street, readiness, school, subscriber, example, guidance, foundling, hearing, preserving, navy, labour, care, term, saturday, management, bench, home, monday, deal, content, order, arc, eminence, taking, making, society (73)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | learner, distribution, carpet, sunday, hook, article, establishment, steadiness, kingdom, thesis, credit, foot, majesty, intuition, promise, store, proficiency, relief, ordnance, surveyor, employment, growth, surname, ease, mechanism, excellence, gratification, support, encouragement, enjoyment, vigour, tackle, manufacture, algebra, discipline, student, philology, interpreting, fidelity, basis, supply, form, advancement, afternoon, perseverance, tutor, compilation, fishing, physics, assistance, limit, trade, road, command, industry, march, advantage, possession, concern, arithmetic, pursuit, science, committee, infirmary, rate, branch, deliberation, syntax, acuteness, future, study, composition, rapidity, profit, livelihood, point, success, anniversary, observation, court, delay, patience, superiority, discharge, friday, vocabulary, mathematician, pleasure, chemistry, resolution, foundation, family, reputation, perception, highness, infant, exercise, opportunity, fineness, provision, chair, residence, teaching, posterity, stress, discernment, council, treasurer, maintenance, hospital, power, person, learning, work, climbing, practice, prejudice, baggage, wood, day, consideration, office, number, mathematics, welfare, time, benefit, tenderness, clerk, master, dignity, revenue, geography, level, importance, subsistence, favour, salary, approbation, contrary, keeper, consent, zeal, acquaintance, authority, geometry, degree, neglect, child, scholar, evidence, duty, goodness, honour, approach, artillery, professor, commerce, pretence, desire, payment, chapel, meeting, doubt, expiration, faculty, neighbourhood, proof, affection, privilege, proportion, knowledge, concurrence, manner, winter, clearness, princess, morning, province, analysis, nature, money, book, claim, compendium, erudition, necessity, reverence, esteem, skill, friendship, name, sum, philosophy, mankind, astronomy, exactness, quantity, wealth, obedience ... (209)                                                                                                                                                                                               |
| +Acad+       | training, tact, value, deduction, patient, scrutiny, estimate, theme, hearer (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | lecturer, cloth, contribution, rote, depth, germ, clue, forte, goldsmith, familiarity, zoo (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | modelling, wisdom, merit, perfection, application, discovery, ledge, inspection, attention (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | slate (1)                                                                                                                                                                                                                                                                         | modelling, lecturer, rote, beginner, germ, sequence, ledge, scrutiny, discoverer, nursery, watt, clue, theme, forte, tuition, deduction, collateral, goldsmith, doc, diploma, patient, information, vocation, examination, aim, drudgery, drawing, attention, travel (29)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | hearer, value, seminary, product, trout, bio, sis, edification, carpenter, merit, weaver, profession, nicety, depth, discovery, observatory, ability, diligence, wisdom, infancy, boy, leisure, harmony, acquisition, maturity, childhood, college, familiarity, weaving, utility, penetration, experience, perfection, dexterity, mason, inquiry, facility, mu, delight, cultivation, subject, sensibility, piety (43)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| +Active      | organization, endowment, conflict, philology (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | urge, partnership, promoter, volunteer, intervention (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | precision, fee, filly, fad, example, preservation, style, instance, safety, merit, posterity, kingdom, zeal, blast, neighbour, proof, constitution, perseverance, honour, ledge, neatness, effect, mutt, dress, study, player, capacity, ornament, attention, performance, esteem, wisdom, assistance, poker, injury, equity, design, support, mechanic, stretch, agency, manufacture, judge, animal, breed, elegance, paper, bulwark (48)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | muff, haft, stile, peak, flue, rake, puss, jade, pie (9)                                                                                                                                                                                                                          | flue, lye, ledge, rally, temperance, tributary, fare, player, hare, drill, furnishing, uniform, directory, legation, urge, lure, endowment, puzzle, toast, rake, elocution, allure, toilet, pie, rating, jade, wage, plea, itch, preservation, simile, expert, stretch, mechanic, partnership, criterion, equipment, preferment, firm, display, paint, tennis, harlequin, exit, fore, joke, intervention, curb, communicating, banquet, dint, task, performer, avail, breed, administration, promoter, example, bulwark, safety, couple, volunteer, system, saving, clown, tuition, strike, labour, attention, applause, trader, printing, game, praise, experiment, union, correspondent, venture, renown, judge, breeding, wit (82)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | puss, steadiness, coalition, naturalist, fusion, fuel, procedure, quill, blast, object, inducement, opponent, ingenuity, neatness, practitioner, scratch, kingdom, tackle, production, perseverance, precision, proficiency, instance, style, talent, effect, knot, caution, fir, merit, profession, knave, shape, imitation, posterity, neighbour, workmanship, brush, derivation, employment, deviation, reputation, ornament, composition, acquisition, compass, dress, compilation, trick, spear, preparation, weapon, repast, mechanism, constitution, instrument, invention, success, industry, understanding, adversary, science, ability, equity, bow, manufacture, support, cure, propagation, poker, magnet, practice, education, assistance, solidity, pen, accomplishment, bribery, sagacity, impudence, combat, paper, foresight, probity, stratagem, honour, tooth, capacity, lawyer, maxim, statesman, philology, harm, munificence, principle, defender, remedy, dexterity, exercise, recourse, attainment, study, advancement, precept, proof, rule, legislature, sanction, skill, abuse, mankind, animal, blaze, integrity, flow, zeal, injury, proverb, fool, mix, adherence, valour, precedent, artifice, elegance, honesty, cylinder, tongue, measure, variety, taste, esteem, wisdom, botany, accuracy, prudence, learning, historian, phrase, fortune, cutting, facility, eloquence, devil, work, armour, definition, testimony, virtue, moderation, ambition, wealth, knowledge, knife, reasoning, judgment (156)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Active+     | suppression, veto, delegate (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | bass (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | manufacturer, series, application, machinery, contrivance, bait, government, ram (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | pattern, belt, stall, strip, mule, pawn, fence, jib, rifle, cement, filter, runner, kite, piece (14)                                                                                                                                                                              | strip, truss, draft, mule, filter, pawn, runner, quadrille, veto, tool, cement, stud, annex, jockey, jib, truck, extension, delegate, umpire, contrivance, digest, bait, scrip, competitor, transit, sewer, challenge, suppression, trot, team, tug, shift, mining, substitute, section, executive, conveyance, publishing, band, guide, fencing, manual, heating, apparatus, cost, inventor, fiddle, survey, management, piece, maker, price, defending, giving (54)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | distribution, establishment, puff, bang, hook, file, tram, spin, formation, fob, slide, kite, crack, twist, para, steel, coke, string, farming, baking, government, bribe, booth, pack, gut, sweep, spade, algebra, bayonet, pin, purchase, hammer, synopsis, harness, regulation, introduction, undertaking, ram, cleaning, collection, bran, machine, husbandry, arithmetic, grant, nail, ivory, whip, plough, negotiation, parliament, prosecution, geometry, trade, agriculture, vote, defence, publication, manure (59)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| +Ambig       |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | presentment, builder, deputy, preface (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | king, zeal, support, power (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |                                                                                                                                                                                                                                                                                   | presentment, builder, peace (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pe, child, support, king, baggage, gi, blood, deputy, day, welfare, lady, alliance, treaty, heart, pa, poem, church, history, power, hill, zeal, book, volume, preface, sense, religion, knowledge, orbit, plant, fear, grief (31)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| +Ambig+      | career (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | goal, clock (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | safety, succession, accommodation, instance, security, reason, happiness, material, honour, advantage, preserving, effect, government (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |                                                                                                                                                                                                                                                                                   | knight, safety, preserving, report, union, content (6)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | den, instance, government, effect, maintenance, approach, expedition, security, negotiation, genius, advantage, reason, kind, loyalty, honour, declaration, happiness, ground, attempt, confidence, respect, claim, love (23)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| +Changed     | regime, drawback, canning, loathing, annoyance, retention, congratulation, veto, advert, trait, advent, hostility, goodwill, supporter, lunacy, endowment, delineation, viewing, threat, incorporation, cessation, consummation, confiscation, ruler, desertion, privation, boon, semblance, lending, deduction, lunatic, sect, irritability, craving, auditor, imbecility, symptom, scrutiny, licensing, ballot, detention, disaster, compulsion, culture, variance, exhaust, absence, lifetime, ascendant, accord, import, seizure, grace, mover, sneer, agitation, par, suppression, transfer, piracy, taint, formula, manifesto, delegate, irony, patient, theme, destiny, peril, fellowship, chagrin, intrigue, correction, currency, longevity, hearing, solemnity, warning, heroine, abbe, strife, arc (82)                                                                                                                                                                                                        | feud, childe, longing, lecturer, clue, caring, germ, serf, herald, mania, thinker, semi, chef, negro, rote, ache, convulsion, papacy, goldsmith, seer, sig, rite, anticipation, discouragement, dwelling, role, mood, renunciation, presentment, gurney, derby, aid, graduate, commune, forte, cheque, sob, debility, trash, meantime, comp, thrill, closing, enigma, labor, duo, budget, reprobation, remonstrance, wail, communication, bey, turkey, sine, blockade, guild, array, abstinence, breadth, urge, moralist, note, star, banking, choosing, sharpness, peninsula, humiliation, harp, data, forum, haven, recitative, gateway, demolition, partnership, collector, engineer, dissent, labourer, embryo, lodger, retribution, victor, jubilee, ego, lane, workhouse, basil, ending, parentage, arbitration, ringing, gilt, builder, din, volunteer, mourning, vent, intervention, marquess, moor, street, scaffold, vita, obstruction, chronicle, relaxation, beau, monday, reckoning, opus, hunting, divorce, accounting, saturday, tavern, tuesday, wednesday, entry, jail, vein, promoter, gazette, thursday, turnpike, quid (127)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | modelling, tank, ledge, squirrel, darling, helm, tent, cairn, reptile, ice, pail, preservation, log, wick, litter, tarn, trail, peat, grit, seam, mechanic, rug, dolphin, elf, shadow, bath, stretch, player, magistrate, sherry, perspiration, consequence, companion, rib, affair, profusion, bait, leopard, paw, fairy, safety, treasure, doctor, frame, fragment, mote, bulwark, contrivance, occasion, example, herd, breed, way, attention, grove, balance, business, conversation, preserving, search, judge, care (62)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | frill, bouquet, leek, tray, gig, punt, divan, jib, mule, disc, kilt, recess, doll, flue, patch, cider, tress, filter, prow, runner, rill, strip, rake, gear, jade, cone, pie, vest, thistle, cement, zinc, manufactory, rift, chine, pawn, monkey, hat, cottage, room, piece (40) |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| +Collective  | regime (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | sunday, monday, tuesday, goal, friday, wednesday, thursday, aid, park, lane, evening, week, afternoon, coming, saturday, cross, chairman, dwelling, peninsula, dinner, snow, quarter, storm, penny, highway, council, budget, square, tavern, noon, wail (31)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | promise, blast, bed, majesty, empire, wick, fringe (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | coon, filter (2)                                                                                                                                                                                                                                                                  | whore, budget, regime, aid, filter, watering, wail, dwelling, wick, tweed, dole, cricket, ruby, spark, rout, tavern, china, peninsula, lane, birthday, saturday, monday, demand, wednesday, bank, thursday, tuesday, yacht, ball, morrow, home, condition, place, coach (34)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | sunday, blast, froth, board, cup, bout, cross, yarn, promise, cotton, bro, fling, sip, chairman, boom, dining, lot, afternoon, park, primrose, chair, pavilion, ham, square, penny, week, friday, snow, evening, quarter, ilk, chamber, fringe, majesty, eld, flag, field, emerald, archipelago, teen, daylight, day, kiss, coming, bed, tower, ton, knighthood, lap, council, oat, summer, road, cabin, empire, barge, morning, commotion, dinner, night, grant, village, storm, wharf, anchor, month, ship, prison, pension, hay, noon, country, grape, shore, gale, hour, hill, office, door, boat, port, sword, radiance, town, apron, river (86)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| +Collective+ | career (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | cloth, era, league, tide, street, herald, haven, bond, gallery, longing, mall, ebb, ending, october, jubilee, din, february, january, september, zoo, contribution, morn, gladness, gazette, tempest, chill, bid, gilt (28)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | hall, concert, lath (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | rill (1)                                                                                                                                                                                                                                                                          | rill, haze, herald, glee, longing, desert, haven, sheen, mist, gathering, jubilee, fete, hail, directory, spa, lust, din, domain, dating, hull, gem, mining, core, gilt, chili, street, ivy, correspondence, gazette, report, ending, season, vacation, peace (34)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | fen, fur, lath, prom, reek, loo, fount, tun, cradle, lull, turf, store, rye, fart, bio, ebb, foam, continent, lilt, silt, bid, tide, chill, morn, gladness, awl, land, crust, market, autumn, anniversary, tee, march, cruise, tub, sod, wreck, glow, dust, mu, gi, mil, tempest, clime, blaze, july, june, breeze, news, january, august, october, september, sky, february, november, meeting, winter, december, year, voyage (61)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| +Conc        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                   | modelling, tank, ledge, squirrel, darling, helm, tent, cairn, reptile, ice, pail, preservation, log, wick, litter, tarn, trail, peat, grit, seam, mechanic, rug, dolphin, elf, shadow, bath, stretch, player, magistrate, sherry, perspiration, consequence, companion, rib, affair, profusion, bait, leopard, paw, fairy, safety, treasure, doctor, frame, fragment, mote, bulwark, contrivance, occasion, example, herd, breed, way, attention, grove, balance, business, conversation, preserving, search, judge, care (62)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | carpet, herring, cob, article, slit, pea, promise, soot, flight, blast, scalp, mirror, cove, ion, lath, shaft, oar, mat, spray, rim, dress, kingdom, figure, onion, snail, bucket, mug, rat, diadem, gad, biscuit, ornament, neatness, precision, bit, fringe, stack, majesty, wing, beetle, posterity, appearance, peg, neighbour, moth, countenance, merit, instance, cot, concern, protection, carving, discovery, satisfaction, thing, inclination, fishery, light, boa, stocking, security, party, lawn, glove, intention, style, vat, wife, perseverance, glory, cushion, credit, event, relief, world, effect, equity, foot, chemistry, empire, dart, comb, toe, urn, broom, commerce, pleasure, poker, bowl, plumage, government, paper, bed, constitution, answer, end, injury, assistance, prejudice, goodness, honour, copy, fluid, stalk, advantage, ram, animal, notion, foundation, wisdom, support, happiness, king, idea, family, cage, hand, throne, heaven, wretch, lustre, species, person, glass, study, capacity, grease, monarch, manufacture, head, proof, title, father, reason, mercy, perfection, distance, favour, soul, antiquity, resolution, strength, friend, motion, water, blessing, creature, authority, trouble, power, esteem, zeal, elegance, argument, salvation, fire, name, courage, providence, meaning (160)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Conc+       |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                   | frill, bouquet, leek, tray, gig, punt, divan, jib, mule, disc, kilt, recess, doll, flue, patch, cider, tress, filter, prow, runner, rill, strip, rake, gear, jade, cone, pie, vest, thistle, cement, zinc, manufactory, rift, chine, pawn, monkey, hat, cottage, room, piece (40)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | plaid, puss, kite, rind, nymph, curve, lump, equipage, bite, crocodile, sea, spider, kit, morsel, wreath, scroll, spring (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| +Desc        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | sob, mockery (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | precision, world, satisfaction, squirrel, promise, inclination, succession, hall, happiness, biscuit, thing, exertion, intention, posterity, filly, wisdom, wife, safety, goodness, spray, performance, way, glory, neighbour, reason, mutt, darling, heaven, honour, countenance, hive, pleasure, affair, sherry, shadow, conversation, wretch, soul, broom, strength, mercy, visitor, ledge, companion, security, rug, father, bucket, consequence, prejudice, modelling, soot, kingdom, magistrate, cove, idea, throne, equity, notion, cur, rim, authority, esteem, vat, pea, paw, answer, log, mat, glimpse, stalk, head, litter, courage, ice, business, figure, salvation, blessing, treasure, dart, snail, mug, toe, judge, bath, meaning, pail, onion, rib, oar, balance, doctor (93)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | bouquet, divan, doll, pattern, plaid, screen, slate, puss (8)                                                                                                                                                                                                                     | bouquet, sob, demon, divan, squirrel, darling, greeting, modelling, ledge, log, doll, gutter, jar, pail, shadow, sherry, paw, lure, yawn, pool, gorge, bath, jug, rug, litter, thinking, ice, companion, magistrate, pane, rib, glen, doctor, stranger, shark, conversation, brute, applause, affair, trot, safety, consequence, way, treasure, peal, balance, saying, peace, memory, business, judge (51)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | bang, bucket, plaid, puss, pore, cove, froth, oar, hiss, promise, spray, rap, festivity, onion, swing, rim, biscuit, reel, slide, soot, lamentation, mug, rush, countenance, precision, mat, thrush, jest, pea, snail, swearing, thing, ghost, wife, posterity, gust, brood, moment, answer, silence, inclination, figure, satisfaction, neighbour, dart, jumping, dash, curl, hem, kingdom, exultation, equity, cleft, toe, scourge, mane, malice, prejudice, voice, gallop, rage, glory, wretch, world, incense, vat, intention, pardon, laughter, gliding, thankfulness, pleasure, smile, dignity, wisdom, goodness, shame, happiness, heaven, curse, mockery, despotism, honour, passion, soul, blessing, idea, prayer, weeping, broom, reason, father, anger, stanza, stalk, humour, misery, security, reproach, ferocity, worship, temper, strength, courage, throne, tranquillity, affection, understanding, spirit, notion, anecdote, salvation, eternity, realm, resurrection, heart, oath, obedience, tenderness, esteem, flow, contrary, word, mercy, bough, exercise, despair, abuse, love, friendship, bosom, god, head, authority, conscience, mankind, consent, sleep, joy, youth, contempt, hatred, corruption, tongue, wrath, breast, meaning, liberty, indignation, pity, evil, piety, guilt, truth (154)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| +Desc+       |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | sink (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | blast, helm, perspiration, hand, sitting, tarn, stretch, slit, trail, cairn, gad (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | muff, mud, stall, railing, jib, stile, peak, belt, jelly, frill, patch, gig, loon, fence, kilt, leek, cone, punt, moss, tray, tress, haft, cider, prow, rake, mule, strip, recess, rill, bite, runner, kite, rift, thistle, sea, coon, chine, hat, lump, hip, kit, rind (42)      | frill, gig, punt, tress, jib, rill, leek, kilt, patch, trail, strip, prow, tray, mule, wile, rake, runner, haze, cider, thistle, rift, cone, perspiration, tarn, stretch, hum, helm, reverie, recess, chatter, lather, cairn, wink, chine, cub, chime, mister, mist, moving, mire, nod, dive, lute, hat, whisper, throng (46)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | slit, hush, kite, blast, rind, bite, gasp, gad, murmur, rave, moan, lump, cataract, kit, glare, load, git, sea, swarm, beggar, groan, kiss, breath, breathing, sigh, slumber, hand (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| +Freq        | tact, exaggeration, sadness, annoyance, missionary, organization, temperament, utterance, career, fervour, estimate, sphere, absence, delineation, semblance, hostility, conflict, emotion, ruler, allusion, irritation, arrest, exposition, epoch, endowment, strife, boon, regeneration, consummation, carelessness, martyrdom, recognition, lack, disciple, destiny, massacre, extinction, peril, enthusiast, courtesy, agitation, value, suppression, habit, record, modification, rebuke, warning, musician, fellowship, advent, supporter, heroine, comment, sneer, insurrection, lunatic, license, consecration, hearing, verdict, churchman, follower, element, magnitude, correction, interpreter, weariness, theme, currency, rumour, reformer (72)                                                                                                                                                                                                                                                             | burst, anticipation, elder, sympathy, intellect, lecturer, distrust, singer, bitterness, park, width, gallery, manufacturing, labourer, peninsula, calmness, era, mania, storm, earnestness, herald, chairman, democracy, communication, denial, street, echo, rector, remonstrance, closing, barrister, abstinence, sink, workhouse, star, dwelling, household, encore, failure, hesitation, mood, choosing, gladness, negro, scepticism, array, concession, movement, explanation, gateway, contact, comprehension, evening, disregard, reliance, derby, cathedral, agony, october, meantime, banking, flash, priesthood, semi, cleanliness, chill, feature, depth, creed, comrade, believer, consistency, mockery, compensation, vocabulary, goldsmith, terrace, phraseology, stability, justification, region, hierarchy, suit, tempest, lamb, blight, butler, ache, exclusion, data, victor, hunting, dissent, monastery, biographer, painter, countess, abbey, theory, collector, brick, harp, ending, explosion, condemnation, inferiority, snow, beach, link, lace, termination, revolt, fanaticism, moor, thrill, summary, bloodshed, rapidity, chapel, mademoiselle, ride, blindness, retribution, intervention, engineer, coat, vestry, frankness, penny, heiress, bond, chronicle, parentage, priory, prediction, humiliation, familiarity, quarter, basil, gilt, willingness (141)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | promise, satisfaction, hand, world, light, manufacturer, inspection, shadow, interest, companion, perfection, family, sitting, agency, material, fragment, sketch, satin, visitor, chemistry, stretch, shaft, machinery, discovery, mirror, glimpse, father, protection, neighbour, name, fairy, blessing, search, ornament, carpet, conversation, perseverance, manufacture, head, meaning, application, magistrate, doctor, party, friend, assistance, slit, government, salvation (49)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | cottage, recess, screen, pattern, manufactory, strip, rifle, vest, morsel (9)                                                                                                                                                                                                     | mania, meantime, annoyance, boon, recording, desert, lecturer, closing, herald, bachelor, host, anticipation, glee, semblance, consummation, uniform, data, exterior, hostility, endowment, substitute, semi, mystery, ruler, manufactory, occupation, vehicle, strip, demon, notoriety, wanderer, negro, stem, absence, suffering, shadow, fellowship, delineation, fairy, supporter, lunatic, mist, banking, dwelling, assemblage, remnant, recess, collector, pilgrim, agitation, domain, stretch, mood, harp, destiny, choosing, novelty, advent, exclamation, speculation, glen, peril, fete, victor, tailor, masonry, racing, stranger, sufferer, passenger, gateway, antiquarian, vest, dismay, star, proximity, peculiarity, derby, retribution, cottage, correspondence, stake, goldsmith, sport, companion, gathering, bidding, staff, possessor, vocation, chivalry, tablet, nook, nursery, nuisance, moat, lust, furnishing, ache, flora, doctor, deposit, array, pantomime, suggestion, banquet, churchyard, extension, asylum, remonstrance, cabinet, counting, suppression, workhouse, humiliation, manual, vision, job, tiger, periodical, tuition, sergeant, stamp, parentage, joke, stud, refreshment, peasant, pretension, display, sensation, abstinence, communication, speed, section, intervention, score, margin, teacher, theme, guise, legion, champagne, peninsula, sample, basil, guest, transition, fragment, workman, detail, leader, bust, scotch, waiter, thrill, emphasis, usefulness, award, salutation, saving, adventure, cheapness, firm, rescue, apparition, aide, china, waste, trumpet, band, whisper, attire, moving, bible, hunting, warning, shelf, couple, rarity, attendant, guidance, cost, performer, abode, tribe, strike, moor, task, fossil, base, correction, street, opening, participation, engineer, applause, labourer, magistrate, ballad ... (282) | sailing, arrest, establishment, distribution, detection, skeleton, sanctuary, exaggeration, cleanliness, carelessness, identity, democracy, naturalist, formation, mound, courtesy, combination, shaft, store, royalty, reviewer, atlas, romance, opponent, regeneration, fiction, hospitality, mirror, extinction, tract, illusion, twist, creed, indication, trout, comrade, casting, stability, concession, removal, enthusiast, follower, supremacy, hush, fervour, biographer, damsel, suicide, specimen, ghost, chest, activity, peep, modification, rush, carpet, border, elder, lamb, monster, martyrdom, diary, impression, drama, reformer, crack, fulness, dash, proprietor, fashion, sting, hook, illumination, porch, mill, owner, flash, silence, suffrage, echo, assassination, catastrophe, glare, blight, promise, accumulation, license, sphere, bravo, predecessor, churchman, steel, sadness, cotton, baroness, profession, painter, driver, production, chairman, comfort, consistency, comprehension, fishing, delusion, salmon, theory, limit, concealment, injunction, region, personage, chill, massacre, belle, bloodshed, selling, sweep, distrust, pastor, table, believer, perspective, observer, conclusion, slit, laughter, mechanism, mademoiselle, park, devotion, consciousness, distinction, cash, ass, rebuke, disgust, denial, phraseology, cup, student, pilgrimage, destruction, atmosphere, recreation, dialect, exclusion, value, repetition, barrister, horseman, verge, trap, gladness, richness, condemnation, contest, farewell, restoration, irritation, comment, landlord, prediction, explanation, inferiority, ingenuity, bolt, lamp, rascal, likeness, sanctity, unity, model, similarity, instrument, reply, witness, basis, burst, representation, cradle, granite, farming, turf, jacket, revival, daylight, device, contract, excellence, architecture, gulf, amusement, stroke, vocabulary, shock, willingness, barbarism, shape, residence ... (815)                                                                                                                                                                                                                      |
| +Freq+       | case, amount, tone (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | list, cloth, cross, dinner, feeling, rise, sound, square, madame, castle, note (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | dress, instance, style, example, business, study, effect, strength, heaven, series, idea, appearance, attention, glass, soul, figure, distance (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | spring (1)                                                                                                                                                                                                                                                                        | note, rest, capital, race, opera, step, system, drawing, sort, information, school, press, music, class, attention, example, season, business, mark, memory, saying, labour, stage, union, theatre (25)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | sound, object, cross, result, square, space, moment, dress, action, artist, extent, existence, instance, grave, effect, style, reference, rose, figure, education, period, sight, picture, boy, spot, field, scene, collection, expression, appearance, progress, act, possession, library, castle, property, science, voice, practice, glass, stone, village, literature, presence, dinner, professor, idea, study, rate, road, supply, meeting, garden, century, hair, college, sun, heaven, colour, spring, respect, taste, palace, wood, door, strength, soul, description, poet, reading, story, building, influence, distance, difficulty, sister, iron, importance, passage, health, song, principle, evidence, mother, girl, husband, poetry, necessity, experience, tree (90)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Human       | irritation, tact, delineation, drawback, deception, musician, organization, privation, tone, delirium, distraction, layman, confiscation, culture, rebuke, impetuosity, penalty, follower, regime, longevity, lending, threat, sphere, disciple, irritability, abbe, lunatic, interruption, imbecility, churchman, heroine, arc, agitation (33)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | league, communication, burst, feud, mount, brandy, vita, encore, peninsula, clue, mutton, mood, manufacturing, indigestion, blight, goo, banking, singer, goldsmith, barrister, cloth, intellect, meal, pint, terrace, lamb, ride, echo, shoemaker, trash, era, widower, relaxation, moralist, valentine, axiom, ego, rite, tinge, haven, nerve, debility, allowance, mall, star, blockade, entry, reckoning, mile, horseback, region, lodger, contribution, wheat, bombardment, retribution, coquette, garrison, biographer, wail, parentage, termination, role, clerk, channel, meantime, epidemic, promoter, rapidity, apprentice (70)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | animal, magistrate (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |                                                                                                                                                                                                                                                                                   | feud, drawback, watering, clue, bachelor, privation, delineation, banking, drill, culture, circus, fete, mood, regime, role, goldsmith, watt, flora, threat, meantime, wail, lunatic, ven, host, confiscation, moralist, glee, rite, communication, debility, imbecility, publican, lending, evacuation, lodger, retribution, fife, vagabond, dismay, sooth, blockade, ditty, irritability, parentage, nabob, trash, pilgrim, haven, sensation, adventurer, agitation, emphasis, plight, flock, novice, pet, peninsula, relaxation, reckoning, star, hermit, tithe, seaman, longevity, exclamation, magistrate, toll, idiot, abbe, subscriber, traveller, vita, ego, ex, entry, promoter, peasant, generation, monsieur, leader, circle, villa, heroine, situation, season, toil, making, arc, taking (89)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | deception, follower, yeoman, layman, shoemaker, blight, goo, lamb, drunkard, valentine, irritation, indigestion, shopkeeper, governess, seminary, sou, rebuke, churchman, delirium, dupe, distraction, malignity, scoundrel, legislator, tinge, plantation, bombardment, biographer, echo, axiom, talc, continent, suburb, rye, burst, barrister, fishing, region, canal, coquette, solace, farming, brood, allowance, vizier, dealer, ride, mum, housekeeper, rendezvous, sphere, author, owner, impetuosity, widower, brandy, epidemic, beggar, matron, rent, penalty, pastor, atmosphere, cat, fish, trading, nerve, mount, boy, ransom, ascent, flower, exultation, consternation, constable, rabbi, interruption, cape, cure, boundary, disciple, cruise, tobacco, escort, beer, running, liquor, moon, equator, village, apprentice, elector, animal, farm, conception, vale, mutton, impostor, rapidity, prospect, vigour, perception, frenchman, clerk, governor, farmer, sailor, attack, malt, frontier, mathematician, alacrity, channel, fortress, mile, ale, soldier, duchy, oats, rivulet, woe, bay, pint, captain, entrance, army, orbit, harbour, rain, intercourse, winter, wheat, payment, siege, horseback, garrison, month, coast, river, weather (140)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| +Human+      | temperament, enthusiast, career, ruler, supporter, trait, desertion, patient, training, sneer, mover (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | childe, serf, negro, commandant, seer, liar, bey, comrade, sympathy, episode, victor, infatuation, beau, basil (14)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | mechanic (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                   | serf, childe, trait, supporter, negro, seer, desertion, ruler, spy, bey, notoriety, wanderer, mechanic, informant, workman, notary, prefect, tailor, dancer, virtuoso, honor, prodigy, patient, victor, potentate, mover, omen, sire, lope, scribe, basil, attendant, vagrant, sneer, spaniard, elocution, clown, beau, lad, joke, trader (41)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | liar, enthusiast, subaltern, huntsman, damsel, comrade, practitioner, nun, puppy, laird, dragoon, planter, belle, mourner, guile, innkeeper, episode, artist, proprietor, commandant, inhabitant, pirate, livelihood, butcher, infatuation, commentator, rabble, admiral, merchant, stupidity, indolence (31)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| +Indiv       | delineation, exaggeration, deception, auditor, abbe, recognition, longevity, mediocrity, contagion, canon, delegate (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | evasion, theory, incarnation, creed, priesthood, moralist, mildness, refutation, preface, retribution, universality, prediction, comprehension, melancholy, meddling, cleanliness, vindication, summary, data, kinsman, commentary, misconduct, familiarity (23)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | study, notion, magistrate, example, wisdom, fragment, goodness, antiquity, reptile, posterity, providence, perfection, elegance, style, conversation, cur, support, preserving, species (19)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | belt, pattern (2)                                                                                                                                                                                                                                                                 | demon, delineation, recording, job, heft, informant, inoculation, employer, reptile, auditor, moralist, retribution, bachelor, magistrate, data, novelty, stranger, physic, sergeant, operator, juncture, tablet, labyrinth, display, fragment, notary, delegate, margin, generality, abbe, longevity, gesture, commendation, system, performer, vision, rebus, traveller, idiot, trace, conversation, generation, attire, example, groom, preserving, curiosity (47)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | deception, creed, theory, exaggeration, foreman, result, recipe, cleanliness, evasion, specimen, prediction, unanimity, claimant, mediocrity, delusion, imitation, injunction, meddling, epistle, client, samaritan, admiration, derivation, incarnation, refutation, preference, mechanism, universality, source, oratory, genealogy, enumeration, multiplicity, inspiration, conclusion, devotion, logic, comprehension, conformity, parable, device, invocation, personage, surname, abridgment, multitude, excellence, valet, reverend, discretion, allegory, discipline, invention, plan, mildness, obscurity, kinsman, antiquity, rabbi, physics, chastity, depravity, distinction, sanctity, declamation, melancholy, practice, misconduct, divinity, reality, period, solicitude, posterity, rhetoric, priesthood, contagion, adversary, understanding, canon, study, view, devastation, spelling, origin, morality, orthodoxy, description, precaution, boundary, model, counsel, style, arithmetic, probability, qualification, rhyme, notion, mathematician, eucharist, painting, support, heroism, orthography, patience, maxim, variety, diversity, purity, pulpit, madness, recourse, laity, structure, elegance, species, defender, learning, wisdom, goodness, pedantry, lawyer, summary, utility, meditation, discourse, simplicity, humour, ridicule, eloquence, gravitation, genius, tradition, licentiousness, steward, liturgy, experience, prophecy, softness, familiarity, forgery, duration, meekness, dissipation, analysis, scripture, arabic, offspring, vanity, patriotism, wickedness, creation, providence, youth, belief, revelation, philosopher, comparison, barbarity, index, taste, perfection, satire, criticism, fortitude, impiety, terror, mankind, thief, homer, preface, nature, elevation, prayer, contempt, confusion, versification, prophet, etymology, indolence, pronunciation, work, clause, clergyman, erudition, truth, piety, immortality, commentary, forgiveness, principle, anatomy, translator, version, credulity, virtue, manuscript, humility, knowledge, testament, chronology ... (217)                                                                               |
| +Indiv+      | case, deduction, compulsion, absolution, tact, threat, veto, carelessness, annoyance, semblance, trait, rebuke, allusion, hearer, inaccuracy, disciple, archdeacon, modification, formula, irritability, irony, fluency, calculation, comment, ordination, courtesy, layman, scrutiny, learner (29)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | singer, encore, indigestion, remonstrance, perversion, reservation, ego, retaliation, inexperience, reprobation, reliance, choosing, intuition, unwillingness, levity, misrepresentation, phraseology, interpreting, impertinence, thinker, axiom, disregard, explanation, clue, faithfulness, acuteness, rudeness, consistency, accounting, infallibility, justification, paradox, implication, infirmity, inconsistency, quid, inferiority, rash, limitation, brevity, bass, narrowness, forte, prohibition, sharpness, deafness, blindness, rashness, biographer, perplexity, concession, pontiff, organist, baseness, vehemence, mademoiselle, willingness (57)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | fee, visitor, neatness, reason, mechanic, precision, equity, inclination, companion, contrivance, judge, figure, animal, player, authority, attention, merit, argument, instance, exertion, ornament, monarch, doctor, answer, onion (25)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | gig, mule, muff, railing, monkey (5)                                                                                                                                                                                                                                              | gig, veto, spy, mule, monitor, threat, drill, trait, retort, reprobation, annoyance, rating, persona, deduction, novice, monkey, formula, mechanic, player, prefect, thinker, simile, clue, freak, expert, choosing, curb, flaw, umpire, semblance, vehicle, exterior, beginner, remonstrance, scribe, wink, exclamation, holder, compulsion, incredulity, brute, irony, contrivance, doctor, adult, scrutiny, apparition, sharpness, gallantry, forte, irritability, sufferer, companion, remittance, attendant, detail, rouge, aide, abbot, dint, greyhound, executive, equipment, ego, collateral, whim, tribe, pedigree, couple, lotion, prescription, function, gait, minority, incentive, query, patrician, situation, attention, ex, engraver, pencil, cell, advertisement, transaction, plot, defending, inventor, genus, quack, judge, readiness, quid, accounting, victim (95)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | detection, reviewer, hearer, carelessness, indigestion, retaliation, reservation, irregularity, oversight, intuition, ass, courtesy, gradation, layman, noun, unwillingness, ease, legislator, impropriety, suicide, modification, implication, impertinence, ingenuity, ghost, damsel, neatness, dialogue, fanatic, oracle, learner, paradox, onion, ruling, politician, phraseology, axiom, rebuke, perversion, misrepresentation, bias, fatality, sufficiency, indication, consistency, narrowness, pique, identity, biographer, twist, dupe, figure, huntsman, inaccuracy, shopkeeper, similarity, narration, deviation, reproof, singularity, interpreting, practitioner, procedure, soliloquy, vigilance, fitness, comment, steadiness, assassination, roughness, extent, caprice, pleading, consciousness, scoundrel, recommendation, limitation, profile, willingness, inferiority, persuasion, judgement, compass, brevity, expediency, inexperience, disadvantage, concession, explanation, atom, deafness, equity, prohibition, caution, mademoiselle, regulation, preamble, acuteness, rectitude, reliance, ordination, malady, ferocity, faithfulness, diffidence, infirmity, calculation, string, courtier, hypocrite, epigram, convenience, miracle, observation, propriety, quickness, interposition, fluency, imperfection, deliberation, absolution, boldness, sparrow, insensibility, ornament, regularity, mistake, magnet, bribery, mask, coolness, brutality, blindness, preacher, spleen, rudeness, astonishment, friar, gravity, penetration, omission, infallibility, exhortation, parent, uncertainty, timidity, instance, admonition, answer, slander, commentator, artist, disciple, sobriety, clemency, negligence, inequality, rash, certainty, merit, foresight, ostentation, anecdote, antidote, impudence, animal, precision, reflection, agility, decision, metaphor, inclination, meanness, sagacity, grievance, partiality, deity, conjecture, nicety, condescension, alacrity, levity, contradiction, probity, forbearance, fallacy, statesman, protector, deformity, disregard, invective, baseness, discernment, epithet, defect, variation, context, falsehood, analogy, reason ... (308) |
| +Interp      | regeneration, lack, lord, inaccuracy, irritation, penalty, endowment, culture, hostility, consecration, martyrdom, longevity, appropriation, judaism, estimate, accomplice, loathing, boon, modification, disaster, exposition, command, champion, semblance, suppression, verdict, trial, impetuosity, lunatic, extinction, manifesto, art, grace, correction (34)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | childe, labourer, league, park, cycle, cloth, burglary, cross, papacy, peninsula, breadth, tuesday, monday, goal, lane, width, gallery, cathedral, sunday, lace, derby, street, array, negro, wednesday, era, infusion, vita, niece, friday, goldsmith, dwelling, recorder, semi, herald, focus, star, mount, chef, fortnight, alias, singer, feud, explosion, decease, thursday, infirmary, argent, oil, warren, walker, adjournment, strand, viscount, gaol, gentlewoman, builder, plaintiff, unwillingness, encore, chairman, chaplain, perversion, sympathy, poultry, coffee, abstinence, dinner, goo, fox, countess, chapel, rite, app, reliance, tinge, banker, coroner, renunciation, stability, forte, brick, marquess, circuit, earldom, reservation, denial, retaliation, county, acquittal, abbey, gurney, vicar, mutton, tide, inconsistency, longing, square, accounting, rector, sig, february, saturday, castle, butler, elder, list, priesthood, mall, graduate, indictment, debility, coat, week, parcel, distrust, comp, bologna, episode, horseback, hunting, indigestion, exchequer, finale, faithfulness, terrace, note, baker, gilt, convulsion, partnership, clock, opus, fertility, monastery, liver, bey, failure, thinker, tile, bond, corps, exclusion, ire, widower, shoemaker, dean, suit, movement, intemperance, bombardment, freeman, biographer, priory, apothecary, tenant, deanery, afternoon, turkey, intuition, relict, evasion, duke, gateway, grocer, arbitration, vault, january, highway, choir, beau, october, consistency, forum, region, channel, preface, quarter, link, provost, custody, candle, scaffold, barrister, complication, convent, labor, collector, inferiority, haven, depth, organist, earl, valuation, apprentice, summary, limitation, september, tavern, promoter ... (218)                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                   | childe, chef, semi, graduate, boon, herald, negro, feud, papacy, watt, goldsmith, culture, endowment, derby, rite, dwelling, flora, turkey, role, loathing, gurney, forte, hostility, debility, lunatic, labor, meantime, array, informant, guild, sig, longing, star, disaster, breadth, bey, note, comp, convulsion, renunciation, thinker, builder, collector, forum, toilet, gilt, operator, peninsula, collation, jockey, deficit, contingent, semblance, workman, parentage, temp, transit, china, embryo, labourer, media, milliner, arbitration, abstinence, longevity, collateral, partnership, deficiency, gateway, suppression, domain, antiquarian, grace, lane, street, beau, hunting, repair, bar, warehouse, racing, marquess, loan, haven, chili, promotion, suggestion, correction, exchange, stationer, tavern, legion, monday, manifesto, promoter, maker, fencing, villa, tuesday, yacht, wednesday, saturday, vita, relation, resort, scaffold, school, cost, opus, thursday, subscriber, accounting, part, temple, condition, coach, place (117)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | regeneration, appropriation, cycle, cross, fox, stability, lace, goo, infusion, inaccuracy, cotton, shoemaker, unwillingness, thesis, retaliation, pendant, formation, atlas, modification, tinge, sunday, valuation, martyrdom, perspective, liver, park, tenant, gentlewoman, freeman, biographer, reservation, bombardment, warren, accomplice, fresco, focus, baker, observatory, square, complication, adjournment, solving, governess, staple, art, extinction, indigestion, coal, intuition, tine, willingness, chairman, banker, canal, explosion, irritation, bio, judaism, corps, device, champion, field, exclusion, interment, episode, cathedral, perversion, implication, sales, consecration, penalty, region, consistency, walker, gaol, matter, hemisphere, butler, indictment, space, predecessor, recorder, steel, denial, evasion, freehold, coroner, niece, diamond, concession, acquittal, friday, chaplain, inferiority, aerial, frock, elder, workmanship, oil, limitation, rebuilding, fertility, advocate, plaintiff, brick, carpenter, extent, reliance, similarity, ordnance, globe, vault, ire, lens, faithfulness, infirmary, command, compilation, verdict, ore, widower, spot, app, boy, board, aurora, quarter, altitude, tower, enamel, weaver, bologna, distrust, week, viscount, suburb, comprehension, arsenal, granite, collier, division, gown, abbey, argent, afternoon, misrepresentation, fabric, chapel, castle, modem, tide, mason, dean, priesthood, apprentice, barrister, flag, silk, embroidery, lord, accident, chamber, market, apothecary, reservoir, earldom, trial, impetuosity, exposition, pair, vicar, brigade, tic, boundary, colour, number, cape, stat, mount, candle, monument, tile, surgeon, intemperance, chalk, deanery, building, citadel, palace, dinner, inconsistency, road, iron, conception, imperfection, county, lease, rector, channel, prevention ... (306)                                                                                                                                                                                                                                                                                             |
| +Interp+     | transportation, tact, canning, delineation, organization, congratulation, regime, retention, deception, drawback, missionary, recognition, archdeacon, ruler, lunacy, sphere, amount, incorporation, career, case, consummation, training, trait, ballot, craving, absence, veto, viewing, supporter, lending, value, dealing, variance, advert, synod, licensing, exaggeration, cessation, confiscation, magnitude, stress, critique, exhaust, allusion, philology, musician, deduction, sect, scrutiny, compulsion, enthusiast, piracy, warrant, par, record, impediment, desertion, symptom, abatement, archbishop, calculation, arrest, goodwill, auditor, canon, intrigue, conference, transfer, heroine, learner, follower, mover, destiny, currency, formula (75)                                                                                                                                                                                                                                                  | lecturer, communication, diocese, clue, manufacturing, aid, bishopric, anticipation, germ, blockade, tariff, engineer, executor, feature, contribution, council, commissioner, contentment, cleanliness, banking, hardship, urge, demolition, painter, mania, surveyor, divorce, data, envoy, gazette (30)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                   | canning, regime, retention, trait, drawback, lecturer, clue, veto, advert, germ, congratulation, aid, consummation, supporter, delineation, viewing, lunacy, ruler, incorporation, sect, blockade, scrutiny, lending, licensing, mania, goodwill, anticipation, confiscation, communication, transfer, absence, ballot, banking, award, engineer, urge, mining, variance, data, cessation, craving, par, occupation, mover, desertion, rarity, piracy, demolition, deduction, intrigue, auditor, symptom, formula, exhaust, tuition, periodical, cheapness, destiny, digest, cabinet, capital, compulsion, currency, inventor, divorce, survey, report, experiment, heroine, gazette, class, emergency, order (73)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | deception, cleanliness, fusion, arrest, learner, tariff, follower, enthusiast, sphere, bishopric, magnitude, exaggeration, value, painter, commissioner, expansion, surveyor, committee, impediment, richness, diocese, critique, purpose, stress, philology, hardship, archdeacon, recovery, astronomer, calculation, viceroy, executor, architecture, council, abatement, student, canon, ratio, artist, synopsis, cardinal, journal, commission, work, supply, contentment, embassy, church, account, pursuance, synod, mineral, archbishop, consul, description, index, duchy, epitome, colony (59)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| +Judg        | exaggeration, privation, organization, penalty, mediocrity, mover (6)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | unwillingness, prologue, tea, levity, penny, banking, punch, brevity (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fee, occasion, performance, affair, consequence, intention, thing, design, material, mutt, reason, pleasure, example, instance, fad, argument, paper, business, creature, credit (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | rake, muff, piece (3)                                                                                                                                                                                                                                                             | rake, lye, privation, mover, twill, fare, substitute, toast, workman, banking, harlequin, consequence, banquet, affair, brute, nonsense, pantomime, joke, task, piece, press, occasion, sage, adventurer, class, bargain, bard, choice, cost, example, venture, readiness, business, content, part, wit, correspondent, praise, place (39)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | epilogue, whit, exaggeration, prologue, puppy, unwillingness, mediocrity, bravo, cake, drama, talent, punch, quill, matter, malignity, impropriety, propensity, frolic, fashion, company, gin, preference, amusement, jest, comfort, paper, author, parody, tinsel, penny, gaiety, brevity, thing, profession, goose, purpose, scruple, lyric, credit, epigram, instance, farce, liquor, temptation, tea, kind, matron, disdain, penalty, bot, indulgence, refinement, desire, humour, gentleman, court, partiality, rule, taste, conceit, foreigner, morality, pleasure, intention, manner, undertaking, pudding, loss, work, disposition, ridicule, pride, maxim, madam, admirer, sex, villain, rhyme, market, scorn, passion, devil, precedent, odium, invective, woman, malice, amendment, goddess, iniquity, fault, reason, writing, creature, entertainment, levity, food, word, attempt, crime, vanity, versification, subject, people, man, temper, refusal, patience, song, misfortune, delicacy, freedom, shame, reader, opinion, principle, mirth, judgment, absurdity, modesty, cutting, sentiment, pound, winter, writer, punishment, meat, mankind, treason, money, severity, truth, obligation, book, nobleman, innocence, vinegar, argument, censure, affectation, injustice, error (142)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Judg+       | sacrilege, champion, intrigue (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | trash, mutton, blasphemy, impertinence, hardship, infidel, coquette, beau, baseness (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | honour (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |                                                                                                                                                                                                                                                                                   | opium, greed, trash, fop, commoner, sooth, chivalry, intrigue, gallantry, beau, diet, breeding (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | drunkard, impertinence, scandal, bully, sacrilege, self, coquette, knave, patron, sloth, champion, slander, fool, comedy, adulation, ostentation, infidel, rogue, hardship, blasphemy, libel, dissipation, drinking, coward, mutton, abuse, baseness, disgrace, licentiousness, hypocrisy, probity, meanness, flattery, fraud, arrogance, impostor, insult, decency, honour, impiety, decorum, beef, folly, contempt, calumny, extravagance, plunder, virtue, indolence, ingratitude, liberty, honesty, moderation, generosity, cruelty (55)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| +Man         | craving, loathing, habit, veto, layman, strife, courtesy, disaster, recoil, intrigue, interpreter, churchman, theme (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | goal, mutton, partnership, bass, echo, burglary, feature, recorder, chairman, chaplain, dinner, vein, apoplexy, rash, provost, flash, din, ear, earldom, region, crew, deputy, clerk, dollar, snow, pang, familiarity, embryo (28)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | neighbour, wretch, squirrel, dart, snail, beetle, mechanic, sherry, bed, bowl, plumage, peg, grove (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | cider, spider, filter (3)                                                                                                                                                                                                                                                         | temperance, filter, veto, cider, wink, employer, strait, pilgrim, hock, desert, mechanic, loathing, spark, fare, glee, toast, fop, partnership, vagrant, squirrel, champagne, sham, remnant, gait, race, lad, sherry, gallantry, adventure, disaster, theme, craving, embryo, job, intrigue, guest, banquet, domain, couple, sweat, bull, vein, fiddle, whist, paint, flute, pageant, din, pencil, victim, seaman, chance, sage, pulse, swimming, antagonist, mien, throng, vacation, grove, toil, breeding, retirement, walking, inventor, chaise, sway, strife, wit (69)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | bitch, tape, rack, opponent, legislator, thrush, recoil, muck, twist, vineyard, layman, planter, snail, dollar, trout, hospitality, beetle, senator, courtesy, glare, frolic, twain, dupe, dart, echo, spider, mouse, churchman, apoplexy, dragoon, pinch, bubble, invader, plume, falcon, peg, swing, moan, perch, merriment, toad, dice, palate, astronomer, bowl, vermin, flash, grave, bush, pigeon, relish, quiver, goose, interpreter, mast, plumage, chairman, mate, neighbour, knot, region, foam, sweep, parody, boom, pang, groan, laughter, ear, spleen, guile, crest, bailiff, repast, ape, string, torch, chin, turtle, wretch, englishman, rigging, propensity, thirst, rudder, complacency, thicket, footman, mathematician, leap, crowd, beak, brutality, bow, sparrow, drinking, chaplain, commentator, boy, forest, recorder, smile, dialect, strain, humour, livelihood, jeweller, box, adversary, owl, rash, snow, stature, scorn, critic, flesh, jaw, visage, manhood, triumph, admiration, keeper, throat, fortune, fate, dust, derision, disdain, adjutant, rage, cat, meteor, tribute, cow, deck, bird, shield, dwarf, member, fowl, blaze, flame, curse, mutton, warden, forehead, mayor, accident, breath, provost, clerk, pride, candidate, acquaintance, bed, bag, hedge, frenzy, harvest, barn, delight, wound, brook, farm, hunger, estate, pair, deputy, grandfather, familiarity, dissipation, path, rebel, frost, gale, nose, temper, appetite, oak, gleam, age, elbow, historian, crew, earldom, youth, cask, bough, trunk, nail, pine, pile, dinner, retinue, eye, noise, borough, brain, attorney, gloom ... (231)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Man+        | canning, musician, ruler, career, supporter, archdeacon, follower, sneer, conflict, delegate, enthusiast, champion, reformer, disciple, mover (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | lecturer, labourer, thinker, goldsmith, negro, fox, banker, shoemaker, comrade, retaliation, builder, hunting, horseback, grocer, moralist, rider, bey, butler, apothecary, serf, apprentice, coat, biographer, painter, slaughter, ride, highway, walker, barrister, graduate, gaol, executioner, derby, indictment, tenant, nerve, victor, moor, punch, bearer, promoter, kinsman, mob, organist, bacon, tavern (46)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | blast, helm, reptile, player, herd, poker, breed, leopard, bait, posterity, animal, monarch, trail, log, cob, ram, friend, paw (18)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | gig, mule, punt, rifle, monkey, runner, kite, crocodile, bite, hip (10)                                                                                                                                                                                                           | canning, mule, gig, punt, lecturer, thinker, supporter, draft, watt, goldsmith, publican, baton, serf, graduate, reptile, umpire, negro, trail, ruler, runner, circus, factor, workman, player, monkey, hare, miner, jockey, bey, greyhound, host, moralist, spy, truck, competitor, helm, neigh, cricket, team, stud, vehicle, log, waiter, passenger, tool, discoverer, tailor, otter, sport, prefect, leopard, racing, delegate, tally, staff, virtuoso, mister, brute, stab, gull, mover, victor, crow, joke, sergeant, watchman, stake, bait, miser, connoisseur, expert, brand, fife, gelding, shark, hound, clown, flock, trump, dutchman, axe, elk, herd, operator, builder, breed, curb, tiger, repute, tribune, band, derby, antiquarian, adventurer, trumpet, trot, foe, knight, wage, hunting, labourer, trader, loading, commoner, vagabond, piper, fist, leader, dilemma, speed, fugitive, moor, steeple, sneer, tribe, paw, traveller, duel, citizen, spectator, groom, fencing, rout, challenge, beaver, hermit, guide, strike, carrier, promoter, tavern, bard, squire, quack, galley, game, clan, halt, mark, maker, felon, fraternity, luck, engraver, renown, aim, coach (147)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | reformer, follower, bang, shoemaker, fray, hobby, naturalist, cob, rider, pun, razor, file, fox, leech, boar, grin, huntsman, buckle, quill, comrade, stag, bully, puppy, hook, retaliation, lance, driver, scoundrel, flint, laird, butt, horseman, ass, yeoman, shopkeeper, bowling, kite, drunkard, blast, biographer, foreman, bravo, bolt, mill, crack, gap, duck, ranger, tenant, kicking, buck, horn, booth, rascal, caliph, turf, trophy, rod, crocodile, bearer, crook, kennel, madman, painter, politician, rabbit, lion, dealer, robber, howl, enthusiast, carnage, livery, spade, practitioner, sexton, stroke, landlord, banker, assassination, weaver, tackle, punch, personage, bite, scratch, cock, hawk, parade, mallet, chariot, monster, dash, bayonet, predecessor, deer, elephant, mace, hack, shepherd, pack, hunter, student, beggar, chaff, pig, observatory, courtier, frog, patron, ox, snake, heron, dick, hog, dun, tanner, gunner, boot, carpenter, drum, champion, fork, butcher, jumping, executioner, rogue, brace, woodcock, butler, volley, coachman, knave, dragon, apprentice, quarry, belly, caravan, trick, contest, pheasant, chop, scourge, swearing, brush, cutter, carver, spear, innkeeper, gang, gallop, corporal, skull, hammer, statesman, hypocrite, coward, mane, shirt, ram, indictment, countryman, limb, jacket, poker, combat, villain, bacon, ensign, collier, trap, constable, slaughter, swine, fishing, apothecary, steel, ride, fool, beard, whirlwind, roar, barrister, frenchman, heel, weapon, kinsman, cane, impudence, benefactor, warrior, whip, preacher, rabble, distinction, killing, valet, peer, dagger, lightning ... (325)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| +Neg         | threat, disaster, massacre, allusion, inaccuracy, semblance, intrigue, warning, visitation (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | burglary, perversion, discouragement, inconsistency, papacy, refutation, levity, rudeness, disregard, disapprobation, unhappiness, trash, vehemence, panic, slaughter, narrowness, tempest, agony, antipathy, humiliation, infidel, reprobation, impeachment, explanation, atheism, apoplexy, chill, termination, earthquake, paradox, violence, penitence, infirmity, robbery (34)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | inclination, security, safety, thing, occasion, succession, throne, reason, intention, neighbour, king, posterity, bed, monarch, idea, concern, glory, equity, instance, prejudice, mercy, argument, notion, fragment, performance, magistrate, event, exertion, preservation, protection, precision, soot, empire, answer, countenance, end, stretch, frame, resolution, judge, meaning, zeal, soul, bulwark, agency, providence, constitution, government, cage, wife, rat, log, doctor, courage, paw, elf, conversation, moth, creature, balance (60)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | stile, belt, muff, jelly, moss, slate, peak, crocodile, bite (9)                                                                                                                                                                                                                  | threat, discouragement, incredulity, humiliation, stab, disaster, papacy, mystery, gutter, log, stoppage, semblance, stretch, blackness, plea, reverie, puzzle, magistrate, reprobation, bulwark, doctor, elf, trash, speculation, paw, intrigue, protest, prevalence, fragment, system, miser, preservation, frame, shark, juncture, administration, safety, felon, nonsense, gesture, whisper, adventurer, tiger, defending, gaming, step, balance, warning, memory, occasion, bourbon, riot, conversation, transaction, crisis, state, dissolution, intent, judge, correspondent (60)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | coalition, howl, gust, accumulation, rack, opponent, froth, kicking, carnage, impropriety, refutation, charge, progeny, narrowness, crocodile, panic, scar, propensity, perversion, caprice, soot, ghost, disapprobation, rat, scandal, rush, bite, lament, jest, massacre, drowning, inaccuracy, contest, shipwreck, repetition, parody, rhapsody, deviation, removal, den, venom, unhappiness, commonwealth, merriment, answer, apoplexy, maze, existence, temptation, vermin, equity, detriment, thing, vigilance, pique, whirlwind, groan, inclination, moth, fable, circumstance, unanimity, protection, neighbour, government, concern, paradox, inability, exultation, infidel, antipathy, obscurity, jargon, prejudice, uncertainty, mandate, chill, rudeness, instance, conclusion, laughter, indulgence, countenance, impudence, extent, event, toad, reproach, visitation, posterity, impeachment, humanity, caution, confederacy, rule, inconsistency, disadvantage, omission, security, conflagration, detestation, dissipation, affront, depravity, clemency, scene, slaughter, revolution, trap, timidity, heap, interposition, explanation, consciousness, deformity, idea, deed, bed, prevention, frog, artifice, king, fallacy, expediency, passion, impunity, despotism, instigation, diminution, monarch, glory, popery, meanness, throne, paganism, resolution, pardon, rag, disregard, end, tendency, dagger, rabble, abdication, ague, revenue, intention, infirmity, amazement, disguise, levity, partiality, plague, redress, scruple, murder, inequality, volley, reason, neglect, abhorrence, scorn, precision, idleness, temper, fright, wife, tumult, delay, decay, bill, recourse, felony, presumption, killing, pestilence, empire, villain, adulation, indifference, cage, legislature, commotion, presence, tempest, arrogance, torrent, adherence, dungeon, adversity, behaviour, crying, englishman, realm, belief, reign, notion, contradiction, defect, beak ... (324)                                                                                                                                                                                                                       |
| +Neg+        | sacrilege, deception, desertion, annoyance, privation, strife, peril, penalty, relapse, recoil, confiscation, detention, accomplice, wretchedness, menace, imbecility, mortification, blunder, weariness, stagnation, contagion, disorder, offender, conflict, interruption, chagrin, insurrection (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | retaliation, remonstrance, blasphemy, bitterness, feud, disaffection, fanaticism, intoxication, evasion, rashness, anarchy, revolt, misrepresentation, banishment, mockery, infatuation, blindness, bloodshed, retribution, sob, intemperance, prohibition, pang, schism, impertinence, blight, perplexity, failure, baseness, hardship, misconduct, epidemic, rash, mania, meddling, sink, complication (37)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | blast, consequence, affair, reptile, injury, wretch, perspiration, trail, herd, dart, shadow, snail, spray, cairn, cur, beetle, leopard (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | mud, railing, fence, mule, patch, strip, jib, rake, cone, haft, kite, jade, monkey (13)                                                                                                                                                                                           | demon, mule, sob, trail, strip, feud, reptile, annoyance, barrier, spy, privation, retribution, desertion, rake, mania, patch, imbecility, freak, suffering, plight, clutch, sham, yawn, peril, pallor, poisoning, perspiration, mist, remonstrance, curb, lust, jade, sufferer, jib, confiscation, dismay, brute, tool, brand, cairn, base, shadow, exclamation, cone, apparition, stain, monkey, disuse, rout, detention, mire, consequence, herd, nuisance, inoculation, leopard, chagrin, suspension, affair, servitude, crow, vagrant, exit, deposition, dilemma, vagabond, foe, strike, cell, victim, snare, rescue, idiot, filth, antagonist, strife, waste, hazard, haste (79)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | retaliation, deception, detection, recoil, fanatic, fatality, contention, annihilation, scoundrel, puff, menace, irregularity, bully, blast, kite, bloodshed, drunkard, dupe, grin, glare, rust, hiss, hush, temerity, suicide, malignity, sacrilege, stagnation, evasion, sting, gasp, meddling, delusion, scourge, pang, madman, monster, soldiery, blight, disgust, rascal, misrepresentation, invader, snail, cramp, silence, impertinence, catastrophe, swearing, lamentation, indignity, prohibition, complication, barbarism, epidemic, blunder, snake, destruction, knave, accomplice, mask, rap, dart, disaffection, insensibility, fanaticism, moan, assassination, murmur, blindness, puppy, witchcraft, misconduct, beetle, trick, intoxication, moment, mortification, indiscretion, mockery, slander, relapse, adversary, load, brood, brutality, infatuation, devastation, revolt, profligacy, coward, imposition, complaint, spray, sloth, swarm, malady, derision, heron, frown, bubble, grievance, alarm, blasphemy, frenzy, chaos, hardship, weariness, malice, ferocity, mistake, spleen, disturbance, stroke, desolation, wretch, hypocrite, combat, deceit, infidelity, licentiousness, penalty, rage, curse, offender, iniquity, wretchedness, bribery, disorder, baseness, libel, bitterness, imposture, rashness, beggar, invective, negligence, pretext, sedition, contagion, confinement, falsehood, discord, fool, rash, doom, disgrace, distress, crush, expulsion, intemperance, imprudence, heretic, fate, traitor, superstition, injury, vexation, owl, odium, banishment, obstinacy, faction, forgery, encounter, darkness, shame, terror, torment, insolence, calumny, barbarity, anger, wickedness, dust, risk, drunkenness, disobedience, hell, usurpation, interruption, dread, usurper, immorality, anarchy, tear, confusion, schism, madness, misery, abuse, provocation, roar, displeasure, disappointment, fraud, impatience, infamy, stupidity, outrage ... (279)                                                                                                                                                                                                                       |
| +Object      | canning, semblance, fellowship (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | justification, anthem, condemnation (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | precision, succession, preservation, dress, injury, merit, fringe, rug, design, constitution, biscuit, filly, inclination, example, cushion, carpet, pea, grit, rim, ice, fad, article, glory, shaft, copy, peat, neatness, inspection, stack, protection, comb, ion, application, grease, antiquity, mercy, instance, carving, urn, rib, esteem, plumage, proof, study, title, leopard, elf, blessing, power (49)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | stile, mud, kilt, divan, patch, cone, cement, rind (8)                                                                                                                                                                                                                            | canning, ledger, jar, whore, pallor, grit, kilt, divan, film, raft, toilette, simile, deposition, patentee, pith, preservation, cement, rug, paving, peat, ice, jet, ration, loop, semblance, stem, presentation, patch, sequence, codex, rib, packet, elf, toast, cone, dispersion, leopard, poisoning, lust, fellowship, billet, peculiarity, enlargement, information, function, applause, administration, couple, band, writ, stamp, sentence, examination, bench, character, memory, printing, example, birch, state, praise, saying, society (63)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | carpet, distribution, detection, shaft, rind, rim, fringe, ingenuity, swing, coke, pea, razor, slide, stack, dress, porch, maple, glare, precision, blemish, closet, imitation, epistle, physics, kettle, drawer, chamber, article, unity, sheet, gasp, ion, curl, urn, foam, neatness, plumage, biscuit, prism, interment, merit, ham, fabric, cushion, fibre, condemnation, string, copy, steel, deviation, protection, holiness, carving, rigging, granite, conformity, education, pardon, injury, vase, comb, communion, porcelain, constitution, chair, side, harmony, brush, dagger, request, form, drapery, imperfection, inclination, divinity, fineness, instance, cornice, sanction, commission, fidelity, treasury, shape, muslin, picture, pope, spear, despotism, college, antiquity, invective, kiss, treasurer, tension, sacrifice, glory, grease, page, testimony, church, trough, evidence, presence, blessing, flame, chimney, righteousness, utility, malice, library, vote, pile, university, assembly, justification, integrity, damask, belief, twine, nail, secretary, blanket, apartment, canopy, worship, rebellion, blaze, president, pulp, writing, oration, feast, symmetry, estate, title, declaration, bead, mankind, tapestry, purse, perfume, study, pistol, defence, text, symbol, tenderness, taper, power, freedom, pewter, sermon, esteem, christianity, importance, manuscript, parchment, ceremony, reign, prayer, gospel, veneration, reading, conspiracy, sword, letter, faith, paragraph, lordship, proof, moderation, conscience, sacrament, revenge, charity, doctrine, girdle, mercy, censure, oath, thigh, covenant, death, assertion, religion, piety, justice, sincerity, guilt, obligation (190)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| +Object+     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | rise (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | material, spray, blast, soot, tank, ornament, modelling, ledge, profusion, hand, slit, litter, fragment, paper, diadem, cairn, bed, contrivance, cage, satin, seam, machinery, poker, mat, log, dart, paw, lustre, preserving, trail, mug, glove, lath (33)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | bouquet, jelly, belt, frill, peak, jib, cider, pattern, recess, slate, tray, screen, disc, flue, filter, moss, scroll, strip, haft, rifle, wreath, fence, thistle, zinc, plaid, gear, pie, hip, kite, morsel, loft, manufactory, rill (33)                                        | filter, bouquet, disc, flue, frill, zinc, strip, tray, cider, tank, masonry, recess, modelling, jib, thistle, stew, ledge, haze, reef, pie, font, fan, rill, clutch, truss, trail, gear, litter, toilet, seam, sewer, cairn, sheen, mist, log, pool, shelf, manufactory, suspension, lotion, deposit, gorge, contrivance, statuary, gutter, pane, paw, profusion, core, gloss, shark, fragment, margin, chancel, mosaic, preserving (56)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | slit, tape, soot, packing, blast, diadem, leak, spray, plaid, skeleton, lath, accumulation, kite, tinsel, gust, hilt, towel, gap, worsted, hoop, ornament, action, mug, paper, curling, mat, morsel, portmanteau, grate, scroll, rig, glove, incense, dart, crape, sleeve, uniformity, embroidery, causeway, poker, scarf, wreath, cage, softness, vellum, lustre, bed, torch, essence, radiance, brocade, collar, hand, pocket, pillow (55)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| +Pain        | irritation, transportation, coldness, semblance, lunacy, absence, rebuke, detention, massacre, reluctance, intrigue, seizure, destiny, outcry, bane, bankruptcy, insurrection, chancellor, hindrance (19)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | burglary, dissent, distrust, disregard, debility, banishment, ache, disapprobation, rudeness, workhouse, celibacy, panic, narrowness, refutation, vehemence, levity, misconduct, epidemic, decease, inferiority, reprobation, convulsion, impertinence, explanation, penitence, scaffold, justification, compensation, priesthood, reconciliation, wail, inflammation, antipathy, apoplexy, exemption, mildness, infidel, executioner, earthquake, jail, termination, robbery (42)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | preservation, safety, security, inclination, posterity, fee, happiness, throne, mercy, wisdom, empire, hand, reason, protection, equity, neighbour, goodness, frame, concern, wife, glory, countenance, ledge, exertion, satisfaction, constitution, zeal, prejudice, providence, business, soul, agency, strength, commerce, thing, monarch, succession, idea, heaven, mutt, salvation, authority, father, answer, argument, instance, machinery, government, attention, event, blessing, friend, judge, courage, care, soot, resolution, meaning, end, foot, support, elf, creature (63)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | muff, belt, strip, stile, moss, rake (6)                                                                                                                                                                                                                                          | strip, truss, ledge, debility, wail, semblance, incredulity, heft, ache, lunacy, preservation, firm, plea, rake, curb, convulsion, reprobation, mystery, seizure, dye, deposition, absence, decline, frame, intrigue, elf, deposit, deficiency, wage, detention, lust, workhouse, incline, protest, mist, system, destiny, dissent, extraction, gesture, gutter, asylum, safety, strike, felon, saving, whisper, administration, corpse, miser, step, exit, axe, chance, juncture, repair, nonsense, scaffold, memory, bourbon, summons, transition, attention, restitution, generation, jail, neutrality, drudgery, business, trace, advance, demand, correspondent, peace, navigation, union, sway, judge, care, refuge, curiosity, toil, content (83)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | progeny, glare, narrowness, hush, opponent, grin, shipwreck, panic, suicide, twain, commonwealth, disapprobation, irritation, kicking, propensity, lament, drowning, humanity, epidemic, result, refutation, carnage, blemish, deviation, mandate, extent, impropriety, protection, scandal, impertinence, murmur, unanimity, rebuke, misconduct, employment, countenance, existence, inferiority, detriment, frequency, celibacy, equity, consciousness, caution, apoplexy, renewal, posterity, ease, procedure, government, distrust, answer, respite, massacre, maze, soot, reserve, cave, clemency, circumstance, orphan, mourner, prosperity, ministry, bane, contest, chasm, reluctance, quiver, reproof, client, rule, inclination, interposition, knave, secrecy, fabric, concern, mediation, wife, neighbour, discipline, indulgence, confederacy, adversity, prejudice, coldness, rudeness, cancer, expediency, security, brood, bankruptcy, detestation, antipathy, dispensation, legislature, explanation, regulation, conclusion, timidity, fool, foot, hindrance, exultation, executioner, dissipation, infidel, petitioner, empire, amendment, spine, hypocrite, astonishment, disregard, amazement, protector, instance, restraint, disguise, banishment, thing, impudence, adherence, certainty, fright, meanness, constitution, wave, scruple, event, discord, change, trading, trade, crying, tendency, compliance, throne, paganism, glory, subjection, felony, parent, voice, penitent, belief, defiance, partiality, indifference, forgery, reply, passion, obstacle, jargon, abhorrence, blessing, wisdom, volley, freedom, disposition, property, pride, settlement, idea, friend, community, temper, dungeon, support, satisfaction, soul, plank, blush, patience, pardon, realm, sacrifice, ague, reformation, welfare, presence, precipice, defender, tranquillity, pestilence, babe, goodness, presumption, commerce, reason, contradiction, arrogance, resolution, compassion, duration, decency, scorn, priesthood, mildness ... (373)                                                                                                                                                              |
| +Pain+       | privation, deception, sacrilege, threat, desertion, annoyance, conflict, case, penalty, confiscation, loathing, stagnation, peril, strife, imbecility, checking, inaccuracy, warning, recoil, disaster, extinction, wretchedness, visitation, relapse, contagion, distraction, menace, taint, weariness, mortification, default, offender, correction, agitation, disorder, carelessness, blunder, chagrin (38)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | discouragement, retaliation, papacy, perversion, blight, blasphemy, disaffection, inconsistency, bitterness, remonstrance, revolt, fanaticism, feud, evasion, unwillingness, rashness, anarchy, intemperance, unhappiness, intoxication, blindness, hardship, schism, misrepresentation, mockery, perplexity, pang, retribution, infatuation, chill, sob, bloodshed, meddling, humiliation, baseness, failure, prohibition, infirmity, agony, slaughter, divorce, remission, sink, rash, deafness, intervention, demolition, tempest, complication, violence, atheism, condemnation, obstruction (53)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | blast, reptile, consequence, injury, shadow, wretch, magistrate, bed, perspiration, helm, shaft, bulwark, balance, trail, relief, dart (16)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | mud, railing, fence, mule, haft, slate, crocodile (7)                                                                                                                                                                                                                             | demon, mule, threat, privation, sob, reptile, annoyance, barrier, feud, imbecility, trail, papacy, cess, discouragement, humiliation, retribution, desertion, loathing, plight, pallor, suffering, confiscation, tool, shadow, stab, disaster, clutch, peril, sufferer, poisoning, perspiration, dismay, suspension, strait, stain, base, brute, inoculation, servitude, yawn, bulwark, helm, stake, remonstrance, magistrate, prevalence, exclamation, brand, disuse, mire, partition, dispersion, intervention, stance, agitation, remnant, survivor, consequence, stem, demolition, warning, blackness, balance, reduction, rescue, divorce, correction, dilemma, nuisance, chagrin, filth, taint, rout, victim, obstruction, snare, waste, idiot, burden, cell, antagonist, foe, haste, dissolution, crisis, defending, brink, separation, strife, state, sweat (91)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | retaliation, deception, detection, recoil, blight, fanatic, annihilation, stagnation, fatality, coalition, checking, howl, irregularity, contention, scar, scoundrel, blast, shaft, menace, drunkard, rack, accumulation, rust, dupe, unwillingness, verge, meddling, sting, gust, temerity, extinction, bloodshed, removal, sacrilege, default, invader, bully, visitation, evasion, snake, cramp, pang, delusion, gasp, perversion, silence, scourge, unhappiness, malignity, barbarism, misrepresentation, complication, crocodile, inaccuracy, madman, disaffection, indignity, catastrophe, disgust, destruction, chill, rascal, lamentation, mask, blindness, stroke, insensibility, remission, dirt, prohibition, recovery, charge, rush, carelessness, load, monster, moan, hiss, soldiery, assassination, vigilance, deed, hardship, taxation, temptation, deafness, fanaticism, distraction, malady, groan, swearing, sloth, grave, venom, moment, affliction, revolt, relief, whirlwind, indiscretion, mockery, slander, brutality, intoxication, heap, witchcraft, coward, disadvantage, obscurity, uncertainty, imposition, mortification, profligacy, infirmity, alarm, disturbance, infatuation, penalty, prevention, relapse, vermin, inability, interment, derision, blunder, revolution, contagion, adversary, deformity, complaint, distress, blasphemy, dart, reproach, desolation, combat, spleen, intemperance, deceit, risk, baseness, wretch, fallacy, ferocity, weariness, frenzy, iniquity, frown, wretchedness, infidelity, inconsistency, condemnation, diminution, offender, curse, expulsion, depravity, crush, grievance, slaughter, limb, negligence, bitterness, fate, devastation, rage, imposture, bill, bribery, omission, chaos, licentiousness, rashness, affront, falsehood, equilibrium, sedition, malice, doom, release, toad, shortness, dust, disgrace, decay, rent, darkness, odium, rupture, reparation, marrow, imprudence, traitor, incapacity, terror, confinement, schism, spear, rash, despotism ... (390)                                                                                                                                                                      |
| +Passive     | sadness, loathing, exaggeration, irritation, delineation, coldness, temperament, irritability, regime, distraction, penalty, semblance, privation, craving, symptom, archdeacon, epoch, import, cessation, emotion, auditor, delirium, tact, estimate, imbecility, verdict, abatement, utterance, hostility, mortification, inaccuracy, weariness, visitation, heroine, taint, abbe, irony, amount, tone, patient, chancellor, fervour, stress, archbishop, relapse, chagrin, wretchedness (47)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | ending, rector, brandy, remonstrance, breadth, blight, calmness, thrill, scepticism, feud, goldsmith, coming, vicar, anticipation, exchequer, star, chaplain, mood, melancholy, cross, bitterness, contentment, gladness, chill, unhappiness, papacy, infirmary, inquest, castle, perversion, goal, gallery, communication, inexperience, serf, cathedral, reservation, lamb, peninsula, bond, perplexity, creed, rapture, commandant, gaol, household, gateway, tide, intellect, mockery, moor, brick, singer, workhouse, meal, tuesday, priory, distrust, rudeness, dean, abbey, february, comrade, hurricane, madame, chapel, wednesday, elder, afternoon, intoxication, haven, enigma, agony, din, humiliation, entry, noon, valentine, infusion, convent, ringing, scaffold, blindness, reprobation, clerk, failure, deafness, prediction, evening, marquess, kinsman, channel, impeachment, beach, contact, gazette, believer, tile, horseback, dowager, explanation, pang, allowance, january, assent, saturday, infatuation, reckoning, levity, penitence, lodger, sharpness, september, lint, cheque, garrison, clock, eclipse, quarter, duke, coquette, zoo, wheat, baron (124)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | happiness, satisfaction, shadow, sitting, countenance, soul, head, hand, fragment, foot, distance, father (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | bouquet, room, sea (3)                                                                                                                                                                                                                                                            | loathing, regime, feud, bouquet, serf, privation, greed, goldsmith, imbecility, irritability, delineation, thrill, semblance, pool, nabob, mood, humiliation, anticipation, auditor, cheque, clutch, mystery, informant, incredulity, star, symptom, suffering, carol, remonstrance, shadow, rouge, glen, gorge, cessation, breadth, dismay, papacy, gateway, enigma, craving, hostility, import, sufferer, reprobation, pet, deficiency, lodger, room, lizard, irony, ringing, moor, haven, mermaid, chagrin, adventure, bible, sharpness, ending, workhouse, notary, din, chancel, communication, troop, poisoning, ivy, vacancy, ruby, shark, taint, felon, reckoning, peninsula, whisper, patient, zenith, gazette, lad, monsieur, highland, beginning, entry, foundling, fragment, relation, lying, abbe, scaffold, abbot, heroine, marquess, corner, drawing, summons, villa, message, memory, saturday, inn, wednesday, tuesday, place, state (104)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | blight, exaggeration, thorn, lamb, creed, sanctuary, distraction, effigy, valentine, chasm, guy, reservation, irritation, chill, comrade, cross, profile, sadness, chest, inquest, balcony, visitation, encampment, hush, prediction, fatality, unhappiness, porch, deli, lint, arbour, illusion, lid, delirium, lamentation, melancholy, stil, reverend, dormer, gladness, sea, consciousness, starch, inaccuracy, countenance, diary, marsh, suburb, penalty, regency, shock, scepticism, mortification, interment, pang, exuberance, chaplain, infusion, deafness, hardness, moan, gasp, lair, coldness, exultation, perversion, verdict, enchantment, residue, guile, chamber, roughness, tower, garret, kinsman, coming, mockery, lava, lilt, believer, moon, side, singularity, sexton, rent, weariness, rector, insensibility, thankfulness, pit, inexperience, abatement, hurricane, fresco, brandy, foot, solace, complacency, dragoon, epitaph, gaol, impeachment, allowance, brick, elder, emerald, vicar, coquette, blindness, fervour, pilgrimage, speech, castle, forest, afternoon, ceiling, tobacco, barbarism, assent, desolation, sublimity, dean, malady, cathedral, sight, distrust, infirmary, pastor, calmness, reality, pearl, childhood, dwarf, intoxication, rudeness, sepulchre, contentment, eld, tide, archdeacon, suspicion, explanation, closet, chancellor, expanse, congregation, bitterness, citadel, grave, governor, hawthorn, relapse, rosemary, gate, ridge, fever, perplexity, child, abbey, draught, archipelago, waking, digression, commandant, time, clerk, cellar, solitude, equator, quarter, emotion, crucifixion, slumber, infatuation, tranquillity, vessel, visage, frontispiece, cabin, stress, cliff, imperfection, brow, house, satisfaction, evening, baron, odour, surprise, noon, penitent, marshal, composure, curate, land, imagery, monk, bay, amazement, dispensation ... (409)                                                                                                                                                                                                                                                                                         |
| +Passive+    | drawback, martyrdom, congratulation, annoyance, desertion, trait, stagnation, grace, variance, carelessness, enthusiast, scrutiny, solemnity, allusion, hearing, agitation, interruption, peril, warning (19)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | childe, longing, ache, dwelling, lip, countess, decease, tinge, earnestness, germ, seer, faithfulness, mourning, niece, hardship, sob, depth, discouragement, liter, exaltation, gentlewoman, snow, storm, vault, episode, caring, ebb, earthquake, disapprobation, blasphemy, wail, inconsistency, apoplexy, debility, incarnation, banishment, mania, condemnation, gall, basil, retribution, vent (42)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | interest, perspiration, bed, visitor (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |                                                                                                                                                                                                                                                                                   | drawback, childe, sob, germ, longing, annoyance, congratulation, trait, seer, wail, fog, ache, pallor, haze, dwelling, greeting, desertion, reverie, sheen, bachelor, mist, trance, scrutiny, wanderer, caring, blackness, mania, apparition, grace, debility, sign, perspiration, churchyard, discouragement, film, vegetation, nook, variance, agitation, emphasis, pretension, core, stain, basil, mourning, retribution, salutation, vision, rest, awe, corpse, survivor, peril, posture, solemnity, filth, vent, brink, burden, veil, warning, peace, horizon, home, hearing, angel (66)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | lip, martyrdom, tinge, hitter, stagnation, sting, carelessness, liter, rainbow, soliloquy, ghost, shipwreck, enthusiast, fount, scar, mourner, dirt, nun, raven, gentlewoman, hoar, disapprobation, gust, apoplexy, atmosphere, cradle, gall, hardship, faithfulness, exaltation, episode, rhapsody, impression, snow, vault, drowning, verge, dearth, speck, heaving, overflow, daylight, liking, ebb, cancer, hue, incarnation, niece, alarm, complaint, depth, respite, condemnation, dryness, crust, indignity, volcano, twilight, ague, flower, consternation, countess, leer, abbess, summer, dungeon, wreck, aurora, crying, precipice, sun, communion, interruption, storm, infant, phenomenon, presence, drought, dignity, accident, earthquake, disturbance, blasphemy, decease, fright, blossom, colour, chaos, limb, elegy, inconsistency, night, beard, widow, wilderness, bed, abyss, banishment, bud, firmament, forehead, lark, paganism, dependence, rheumatism, mantle, resignation, pestilence, couch, confession, dew, lady, saviour, pillow (114)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Perc        | archdeacon, training, extinction, detention, canon, philology, verdict, emotion, visitation, recoil, fellowship (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | lecturer, clue, rector, thrill, relict, chant, coroner, flash, infirmary, elder, reservation, blindness, vicar, highway, priory, tenant, list, snow, meal, tinge, lodger, contribution, sound, surveyor, noon, moor, presentment, castle, forum, cheque, entry, earldom, universality, basil, din, ear, heiress, brigadier, familiarity, acquiescence, ward (41)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | study, biscuit, modelling, stretch, glimpse, countenance, stocking, spray, diadem, cot, leopard, bulwark, relief, satin, preserving, dart (16)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | recess, bouquet, railing, manufactory, runner, thistle, strip, vest (8)                                                                                                                                                                                                           | lecturer, clue, bouquet, entirety, draft, recess, modelling, font, runner, thistle, manufactory, thrill, baton, cheque, fog, reverie, strip, cologne, stretch, vest, lodger, presentation, waiter, presentment, fuss, tributary, riddle, patentee, basil, forum, suspension, cornet, detention, loading, codex, roan, leopard, lotion, labyrinth, mystery, exclamation, aide, troop, vision, repository, fellowship, crouch, bulwark, shift, manual, chancel, crow, gem, minster, survivor, beck, moor, tuition, sandwich, tiger, dive, whisper, step, hull, din, intro, rescue, tonnage, navigation, jersey, inn, preserving, pulse, entry, trace, yacht, concourse, horizon (78)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | indication, glare, detection, quiver, tenant, boll, extinction, flash, slide, recoil, maze, reservation, scent, counsellor, sound, prism, diadem, rush, steadiness, speck, activity, enchantment, consciousness, den, effigy, huntsman, lock, tinge, dun, buck, brim, sexton, murmur, acquiescence, gliding, falcon, magnet, dragoon, visitation, turret, coroner, surveyor, blindness, cot, vigilance, daylight, deviation, pennant, ordnance, sight, biscuit, astonishment, elder, peep, snow, bro, stocking, canon, kin, expectation, expansion, baronet, marsh, similarity, universality, slope, spray, countenance, moan, relief, leap, rector, muslin, dart, relict, ear, verdict, git, slumber, ensign, reflection, waking, gaze, approach, philology, breathing, promontory, thicket, sweep, encounter, commander, spot, twilight, hemisphere, visage, glade, infirmary, vibration, expanse, boundary, cap, meteor, commencement, quarry, treasurer, sparkle, gat, study, accession, cutter, grandfather, vicar, settlement, alarm, sparrow, chin, archdeacon, poplar, brigadier, adjutant, dawn, perception, blaze, gleam, smile, desolation, understanding, cliff, eruption, noon, consul, priory, castle, ray, roar, congregation, radiance, maintenance, gloom, familiarity, glow, firmament, structure, phenomenon, crucifix, window, jurisdiction, breath, city, beak, botany, taper, eye, territory, welfare, sky, bough, contemplation, breeze, gravitation, earldom, emotion, cheek, sleep, orb, cavern, analysis, languor, shoulder, frigate, abyss, pasture, pillow, captain, charter, arrival, patriarch, death, shore, river, heiress, orbit, dominion (183)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| +Perc+       | sadness, drawback, recognition, advent, deduction, advert, case, cessation, arrest, conflict, semblance, utterance, scrutiny, epoch, exhaust, consummation, estimate, interruption, accord, formula (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | goal, convulsion, cycle, era, dwelling, distrust, thinker, calmness, anticipation, sink, seer, sob, rise, gladness, chill, gateway, lip, sharpness, germ, haven, closing, intuition, star, ringing, limitation, vault, prediction, agony, complication, pang, hurricane, believer, data, earthquake, beach (35)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | succession, visitor, tarn, cairn, fringe, safety, wick, rug, helm, slit, cove, interest, rim, chemistry, trail, paw, tank, tent, sitting, perspiration, shaft, inspection, dolphin, rib, companion, ledge, seam, cushion, fragment, grit, stack, log (32)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | jib, belt, disc, patch, screen, prow, tress, gig, sea, filter (10)                                                                                                                                                                                                                | disc, advert, haze, sob, drawback, advent, greeting, inlet, trail, convulsion, diver, jar, thinker, raft, pallor, coil, sheen, tress, seer, span, wick, proximity, barrier, film, jib, germ, scrutiny, cairn, prow, clutch, dwelling, stair, pool, stoppage, tarn, tank, patch, mist, reef, consummation, closing, sharpness, gorge, sequence, packet, deduction, filter, blackness, car, gig, anticipation, semblance, glen, flaw, mariner, scan, cessation, paw, lizard, apparition, trance, tent, shark, salutation, rug, stem, baud, pane, ledge, hum, grit, gateway, helm, masonry, accord, seam, rib, wink, recording, dispersion, thud, dismay, jet, haven, log, loop, lighting, ringing, data, perspiration, dolphin, bight, yawn, scat, nod, vacancy, star, formula, marl, nook, churchyard, media, watchman, enlargement, stain, toll, exhaust, ruby, companion, deposit, halt, stoop, chime, shelf, peal, awe, corpse, summons, guidance, safety, brig, fragment, bank, access, swimming, state (126)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | bang, arrest, tick, sailing, slit, gauge, fatality, encampment, skeleton, gap, cove, lip, horseman, scar, swing, profile, foam, concealment, rim, cataract, perspective, shaft, hush, hilt, gust, cycle, porch, keel, prick, rap, curling, intuition, prediction, illusion, fringe, complication, illumination, chasm, lair, rook, leak, chill, primrose, sadness, mound, accumulation, drawer, curl, hiss, lag, creek, rainbow, gasp, observatory, lull, reel, isthmus, limitation, bolt, celerity, twist, distrust, toot, gulf, gladness, astronomer, recovery, boom, pang, ark, stack, bias, removal, quay, heaving, electricity, emerald, portal, grate, twig, sea, lobby, lens, rigging, vault, ferry, cleft, wand, chemistry, trespass, swarm, schooner, lightning, catastrophe, dial, jumping, interruption, tackle, physics, hue, perch, rudder, growth, scarf, deer, landing, synopsis, dispensation, believer, cabin, spine, beech, lava, bridle, attainment, student, tension, canoe, mane, crest, plank, inspiration, continuation, quickness, interposition, calmness, wharf, elm, parapet, caravan, pier, chart, cushion, coolness, chamber, volley, consternation, hurricane, solicitude, ridge, pilot, skull, sod, barge, tower, cough, contraction, ceiling, transcript, discharge, gate, escort, inundation, frontier, shower, beach, canopy, serenity, meadow, interview, geography, wrist, agony, presence, shot, earthquake, anatomy, respiration, map, revelation, intercourse, astronomy, guard (173)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| +Pleasure    | drawback, delineation, fluency, goodwill, musician, consecration, culture, fervour, art, arc (10)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | childe, tuesday, sunday, cloth, friday, goo, cycle, lane, street, poultry, mount, saturday, era, infusion, breadth, horseback, pint, afternoon, mutton, width, meal, goal, chairman, strand, baker, sig, derby, brandy, builder, week, lamb, oil, list, haven, vita, ride, dwelling, rote, engineer, corps, opus, mall, october, butler, gateway, september, warren, commandant, alias, castle, presentment, zoo, marquess, bond, comp, noon, app, chi, aid, walker, square, link, clock, cam, moor, january, dollar, candle, penny, gazette, beau (71)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | biscuit, water, cot, stocking, bit (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | divan, piece, pie, manufactory, room (5)                                                                                                                                                                                                                                          | childe, drawback, entirety, watt, goodwill, largo, divan, font, pie, sig, delineation, toilette, rote, dais, glee, presentment, gelding, quadrille, circus, fern, roan, derby, manufactory, equipment, forme, viol, tweed, tome, bachelor, culture, assemblage, connoisseur, pet, dwelling, stock, masque, warehouse, discoverer, comp, cologne, glen, curry, furnishing, haven, mess, aid, builder, virtuoso, nook, doc, vole, breadth, fife, gateway, engineer, steeple, larder, cornet, terra, corpus, piper, van, waiter, duct, burg, street, elocution, statuary, sport, commendation, scotch, lane, row, saturday, mermaid, lighting, bar, room, birthday, possessor, jam, moor, beau, marquess, nap, stationer, tuesday, ivy, ball, monsieur, opus, gazette, term, piece, birch, swimming, ticket, vita, chaise, arc, coach (101)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | rowing, goo, hook, mill, coo, sunday, lamb, seller, encampment, bio, baker, vert, fur, cycle, sib, infusion, guy, kettle, tine, dollar, salmon, yeoman, bout, canary, fir, art, yarn, loo, peep, corps, oyster, dram, fiction, fluency, bit, recreation, chairman, cock, afternoon, tun, festivity, ham, booth, space, prism, friday, ax, tin, richness, turf, nut, gin, bro, ilk, tilt, edification, biscuit, rum, cot, mare, jeweller, gal, weaver, warren, square, plat, chi, consecration, sty, lyric, filling, butler, mountain, hooker, rosemary, porch, fervour, lens, week, proficiency, tart, ride, commemoration, lemon, soup, boy, cap, tar, tot, carpenter, quay, stocking, hunter, tobacco, basket, creek, pan, composition, cam, tackle, oil, repast, sim, schooner, talent, ferry, gi, penny, inspiration, muslin, fleet, field, tad, wood, colour, cruise, oat, eld, nit, diamond, exuberance, tab, frock, woodland, poplar, barge, mosque, bay, brandy, village, castle, butter, tit, tee, beech, mount, wharf, reservoir, ide, walker, brass, frontispiece, spoon, sparkle, bee, rivulet, damask, ant, noon, app, morning, leisure, lake, pewter, shop, tic, bunch, amber, mixing, mason, mutton, conception, cape, commandant, mu, ivory, telescope, ton, candle, horseback, land, merchant, entertainment, paste, frenchman, veal, pint, gate, beer, june, anchor, drinking, pepper, variety, boat, river, plate, captain, facility, shore, prose, pasture, elasticity, velvet, hay, pair, hill, size, september, cheese ... (227)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Pleasure+   | tact, recognition, lending, lack, critique, enthusiast, trait, exposition, advent, learner, accord, par, training, fellowship (14)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | encore, singer, monday, lecturer, wednesday, goldsmith, park, thursday, chef, dinner, finale, coffee, gentlewoman, tea, contribution, forte, duo, evening, chant, thinker, herald, star, harp, solo, oratorio, jubilee, mademoiselle, gilt, nightingale, gallery, anticipation, madame, punch, choosing, tavern, anthem, guild, graduate, suit, hunting, terrace, vein, role (43)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | sherry, satin, hall (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | bouquet, cider (2)                                                                                                                                                                                                                                                                | chef, bouquet, lecturer, goldsmith, fete, watering, cider, carol, lending, trait, harp, forte, thinker, herald, guild, thrift, prelude, role, sheen, spa, cricket, duo, hock, advent, graduate, sherry, award, mater, gem, accord, stud, tennis, ditty, sprinkling, milliner, jig, champagne, gilt, star, presentation, capital, homo, jubilee, par, cheapness, dancer, salutation, advertiser, anticipation, repute, fellowship, nursery, housewife, drawing, choosing, delivery, racing, whist, lute, tuition, refreshment, adventure, teacher, rarity, periodical, ruby, tavern, prodigy, exchange, ballad, villa, card, monday, vein, circle, wednesday, knack, lily, yacht, spending, hunting, inn, thursday, highland, vacation, information, season, supper, dancing (89)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | holly, hobby, spice, learner, pastry, enthusiast, illumination, dining, sip, peacock, atlas, deli, gentlewoman, mademoiselle, pavilion, pendant, rainbow, naturalist, bowling, plum, pantheon, pic, punch, arbour, combination, park, fishing, trout, primrose, motto, staple, belle, nightingale, store, critique, cake, aurora, tea, rendezvous, emerald, chocolate, company, porcelain, evening, cup, author, pearl, cherry, dairy, gardener, garland, fountain, summer, hawthorn, comedian, flower, cookery, parlour, almond, journal, astronomer, dinner, amphitheatre, venison, exposition, vase, abundance, anecdote, ale, octavo, cream, flavour, meadow, claret, synopsis, anniversary, turtle, mathematician, narrative, honey, embroidery, wedding, dish, comedy, retinue, blossom, firmament, lexicon, orchard, silver, quarto, extract, book, wine, volume, essay (96)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| +Plural      | lord, bane, trait, layman, art, habit, proclamation, delegate, trial, chancellor, sect, visitation, interruption, command, theme (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | communication, cross, renunciation, cathedral, burglary, dinner, lane, dwelling, street, gaol, evening, sunday, pint, wednesday, tile, friday, county, jail, rote, plaintiff, ear, aid, harp, role, preface, cam, infusion, castle, envoy, coat, clerk, brigadier, chapel, promoter, paradox, seer, bid, clock, levity, abbey, entry, rider, bearer, grocer, quarter, surveyor, ban, theory, executioner (49)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | fee, occasion, affair, exertion, hall, merit, hand, concern, dress, thing, article, fad, design, person, proof, head, cushion, reason, throne, water, consequence, authority, vat, attention, example, hive, wick, mercy, king, interest, way, family, cur, advantage, ion, foundation, bed, countenance, stack, father, security, soul, wife, pail, wing, meaning, bit, rat, prejudice, name, tent, mote, copy, bulwark, judge, resolution, end, log, application, cot, toe, friend, lath, empire, concert, reptile, mat, breed, foot, fort, power, title, glory, treasure, conversation, oar, ram, pleasure, rib, bath, style, boa, heaven, light, urn, honour, answer, party, bucket, paper, relief, care, mould, government, fire, balance (96)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | belt, stile, doll, fence, slate, cone, haft, peak, tress, plaid, pawn, room, rill, flue, rind, tray, piece, pie, prow, loft, rake, hat, hip, monkey, sea (25)                                                                                                                     | font, tress, rill, lass, pail, raft, rime, doll, tome, role, flue, rental, pie, trait, cess, renunciation, wick, pawn, celt, cone, jet, log, nabob, retort, effusion, lure, itch, spa, prow, tray, fife, lye, rake, dwelling, rote, harp, wile, bath, tent, liege, fare, pew, hum, fop, scat, communication, car, aid, hare, fist, reptile, rib, lad, dye, fan, span, dispatch, gene, base, mote, allure, seer, bulwark, chime, greyhound, pane, usage, kid, monkey, sect, feu, survivor, rood, uniform, wort, room, delegate, pool, pet, cation, detail, affair, bar, crow, theme, ottoman, collation, breed, posture, halt, dole, stance, rape, foe, reduction, jam, jail, consequence, dancer, gent, sentence, deposit, situation, sign, elk, race, attention, task, cent, milliner, relation, supper, press, loop, hazard, hat, promoter, repair, lane, hail, street, stake, administration, tune, date, intent, service, treasure, bench, truce, bookseller, union, rest, conversation, veil, system, pencil, occasion, home, hoard, piece, memory, example, bank, maker, bust, way, state, balance, ball, report, demand, entry, brig, labour, judge, wednesday, place, society, part, order, wit, venture, temple, content, character, campaign, price, care, coach (170)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | fen, coo, establishment, rind, imp, epilogue, prop, fray, lath, leak, fice, oar, file, bout, yarn, plaid, pun, tort, cross, talc, char, pate, rat, charge, deli, bane, bit, eve, layman, puppy, gel, mace, ax, fart, stack, fem, lag, guile, brood, drama, mourner, wing, article, judgement, recommendation, reed, cad, dud, quill, den, ger, bib, repose, art, rod, invader, ion, toot, rider, caprice, peerage, bucket, rant, cot, freight, pap, cert, caw, coin, phi, teen, bearer, lens, ham, cud, ghost, lam, soliloquy, mat, dress, continent, bottle, blemish, boa, lull, specimen, grotto, dun, concern, roller, sunday, cue, paradox, gal, merit, ear, fling, ox, cushion, hue, dice, imitation, countenance, cock, touch, ease, cathedral, vat, pursuit, fabric, foot, fleece, urn, toe, loo, gram, visitation, proclamation, matter, infusion, reel, mead, sight, consumer, fable, scandal, cam, source, cop, dock, cat, thing, lime, mare, tab, gaol, bow, gat, ram, lab, party, gang, bat, rad, dial, marshal, bid, pin, deliverer, amp, copy, reality, speech, tincture, abridgment, evening, committee, narrative, court, epistle, house, root, board, purchase, author, answer, page, rupture, ban, chariot, relief, tau, folio, tee, ore, prospect, review, teat, computation, surveyor, ton, moo, valet, plunge, augmentation, ape, collection, theory, ray, loin, university, act, wife, deceit, prejudice, pen, portal, legislature, deed, cap ... (530)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| +Plural+     | auditor (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | gentlewoman, bishopric, rite, broker (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | pea, tank, fishery, moth, paw (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | stall, recess, kite, patch, nymph, filter, chine, thistle, rift, equipage (10)                                                                                                                                                                                                    | filter, fern, tank, recess, tuft, thistle, flaw, ration, strait, rift, clutch, lizard, patch, auditor, chine, playhouse, temp, bale, rite, physic, dive, gelding, troop, tiling, paw, pant, denomination (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | kite, scar, pea, sou, pear, woodcock, gentlewoman, fishery, hoop, windmill, fib, cleft, nymph, parody, propensity, jeweller, moth, gnat, poplar, rook, equipage, bra, pill, bishopric, broker, toad, cadence, sparrow, scruple, elector, inequality, couplet, butterfly, inundation, reservoir, negotiation, orb, barge, dividend, taper, prelate (41)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Polit       | irritation, loathing, confiscation, irritability, hostility, organization, allusion, taint, threat, delirium, case, seizure, coldness, conflict, sneer, desertion, sadness, ballot, menace, insurrection, rebuke, piracy, privation, proclamation, symptom, consecration, advent, synod, magnitude, bane, peril, manifesto, abbe, outcry, delegate, chagrin (36)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | semi, league, bloodshed, blight, encore, infusion, democracy, prediction, mania, creed, banishment, arbitration, scaffold, anarchy, bombardment, episode, epidemic, intemperance, abstinence, vindication, panic, dissent, acre, echo, ego, sob, revolt, finale, jubilee, fox, prologue, coquette, inconsistency, quid, slaughter, storm, parcel, impeachment, misrepresentation, misconduct, flash, executioner, envoy, impertinence, indictment, chill, pontiff (47)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | filly, spray, broom, dress, reptile, stalk, agency, machinery, stack, fairy, glass, fragment, soul, idea, poker, bulwark, ram, paw, plumage, bowl (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | jib, stile, bite, wreath, piece, chine, spider (7)                                                                                                                                                                                                                                | loathing, semi, mania, threat, confiscation, jib, fete, sob, assemblage, seizure, draft, reptile, opium, hostility, irritability, quadrille, tribune, masonry, jockey, desertion, privation, evacuation, incredulity, arbitration, advent, fairy, baud, wanderer, reef, harlequin, freak, taint, fist, transit, tablet, vegetation, apparition, gelding, spectator, necklace, sheen, emphasis, exit, piracy, chine, pane, inoculation, opera, dais, ballot, peril, fan, bight, mess, paw, jubilee, dutchman, masque, troth, chagrin, symptom, playhouse, delegate, applause, job, gesture, axe, whim, sensation, bulwark, sneer, abstinence, scaffold, margin, gaming, squire, bull, abbot, brute, china, stranger, truce, felon, manifesto, race, galley, abode, dissent, ego, system, fragment, peasant, dissolution, domino, attire, adieu, tale, abbe, victim, piece, sway, quid, horizon, advertisement, dancing (105)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | coalition, bloodshed, prediction, menace, claimant, blight, democracy, bite, creed, fiat, bombardment, invader, puff, grin, enchantment, gaiety, drug, spray, infusion, parody, drunkard, merriment, soliloquy, epidemic, irritation, panic, delirium, fox, soldiery, maple, identity, impropriety, rascal, hiss, stack, belle, hermitage, brim, prologue, damsel, tape, demonstration, bane, armada, coquette, proclamation, pantheon, drowning, finery, area, bolt, interlude, rebuke, royalty, indemnity, fable, dress, bubble, episode, bier, canton, thrush, abbess, politics, republican, devastation, motto, propensity, clemency, drama, resistance, flash, impertinence, stroke, magnitude, dash, cock, bark, farewell, echo, farce, spleen, reply, instigation, dwarf, bottle, misconduct, buff, courtier, executioner, heroism, pit, shape, misrepresentation, booty, spider, pheasant, cession, impeachment, valve, cup, emerald, causeway, contest, sadness, chill, hypocrite, viceroy, mandate, fabric, hue, plumage, infection, rabble, appellation, witchcraft, expulsion, lyric, bribe, knave, existence, frontispiece, abdication, insurrection, odour, consecration, ram, digression, accent, chaos, sanctity, traitor, epigram, mane, indictment, bowl, soil, broom, faction, exuberance, slaughter, sheet, monopoly, hemisphere, song, coldness, meteor, dance, destruction, poppy, knot, intemperance, revolt, falsehood, muslin, usurper, banner, tragedy, dissipation, eruption, murder, invocation, malady, venison, rage, sleeve, sulphur, protector, collection, hell, sun, stanza, brocade, banishment, jacket, independence, stalk, deceit, grape, coachman, owl, commotion, couplet, oration, drinking, emblem, symbol, swine, soup, paradise, amphitheatre, wedding, visage, blanket, bondage, desolation, acre, poker, wreath, forgery ... (339)                                                                                                                                                                                                                                                                                                                                                  |
| +Polit+      | canning, ruler, retention, regime, semblance, veto, disaster, strife, accomplice, consummation, incorporation, stagnation, sacrilege, intrigue, mover, agitation, compulsion, massacre, champion, suppression, disorder, wretchedness, imbecility, contagion, destiny, recoil (26)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | retaliation, feud, disaffection, convulsion, burglary, remonstrance, serf, bitterness, blasphemy, intoxication, papacy, burst, concession, explosion, hurricane, partnership, peninsula, fanaticism, evasion, negro, budget, kinsman, termination, hierarchy, retribution, baseness, argent, turkey, gateway, mockery, humiliation, intervention, infatuation, tempest, stability, reprobation, robbery, obstruction, comrade, condemnation, moor, highway, rash, ban, commune, earthquake, vehemence, antipathy (48)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | blast, wretch (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | belt, strip (2)                                                                                                                                                                                                                                                                   | demon, serf, legation, feud, canning, budget, veto, regime, convulsion, retention, stoppage, spy, rally, turkey, ruler, consummation, brand, strait, disaster, deficit, commune, humiliation, mover, semblance, plight, retribution, strip, louse, confederation, remnant, blackness, incorporation, sham, papacy, negro, agitation, intrigue, toast, remonstrance, imbecility, guise, reprobation, intervention, stab, rental, partnership, rout, chivalry, bidding, tartar, gateway, nuisance, obstruction, inauguration, suppression, gallantry, destiny, shift, deposition, duel, stake, moor, transaction, compulsion, salute, tiger, banquet, peninsula, pageant, adventurer, challenge, tribe, patrician, citizen, omen, antagonist, fraternity, throng, foe, strife, peal, clan, crisis, separation (84)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | retaliation, fray, fanatic, bravo, stagnation, festivity, malignity, carnage, scoundrel, concession, opponent, twain, recoil, glare, howl, senator, assassination, border, planter, accomplice, suicide, blast, monster, stability, rack, catastrophe, annihilation, bully, disaffection, gauntlet, robber, whirlwind, foam, suffrage, lamentation, evasion, comrade, scourge, capitol, ruling, mask, kinsman, swarm, sacrilege, ban, hurricane, champion, gust, exultation, massacre, swearing, conflagration, dragon, pavilion, explosion, condemnation, intoxication, indiscretion, respite, hospitality, deed, portal, hierarchy, bride, plume, barbarism, vermin, fanaticism, trophy, chariot, laughter, heretic, dungeon, slander, frenzy, madman, disorder, sweep, pirate, rose, baseness, mockery, ferocity, scarf, infatuation, curse, invective, ferment, brutality, scene, derision, volcano, burst, blasphemy, bitterness, detestation, bridegroom, adversary, defender, doom, uproar, iniquity, contagion, libel, torch, dagger, promontory, wretchedness, antipathy, patriotism, smoke, sack, rampart, rash, blaze, prospect, combat, meanness, ermine, coward, wretch, villain, warrior, barbarity, stratagem, shield, liquor, adultery, vow, patriot, tempest, grenadier, pestilence, cardigan, pride, disguise, argent, calumny, adulation, shame, anger, frown, spear, cloud, gale, weapon, marchioness, flame, murderer, arrogance, hanging, wickedness, madness, velvet, resignation, drunkenness, eagle, nobleman, pressure, roar, prelate, imprudence, wrath, plague, earthquake, bigotry, cloak, excommunication, robe, thunder, ingratitude, impiety, vehemence, idolatry, death, neck, coronet, helmet, azure, generosity (180)                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| +Pos         | drawback, delineation, sphere, congratulation, culture, goodwill, fluency, exposition, amount, grace, fervour, art, supporter, fellowship, estimate, longevity, churchman (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | cloth, cycle, sunday, tuesday, lane, monday, breadth, goal, wednesday, friday, mount, cross, labourer, park, manufacturing, intellect, pint, street, chant, thinker, thursday, childe, league, infusion, coming, width, fortnight, coffee, faithfulness, tea, strand, contentment, star, focus, dwelling, horseback, afternoon, derby, oil, baker, meal, saturday, october, sig, haven, circuit, brandy, poultry, lamb, negro, parcel, evening, app, rote, gallery, mutton, lace, mall, fertility, walker, marquess, warren, dinner, mile, butler, eon, anticipation, sympathy, september, link, corps, bass, banker, earnestness, terrace, square, ride, quarter, clock, tile, widower, finale, suit, cam, seer, county, opus, willingness, january, coat, sharpness, week, mademoiselle, expiration, dollar, shoemaker, chi, universality, depth, liter, acre, penny, priory, role, acuteness, chapel, commandant, noon, castle, candle, cheque, stability, beach, apothecary, ling, believer, grocer, deportment, contact, punch, lodger, deacon, zoo, embryo, vein, channel, wheat, morn, tavern, auto (130)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | sherry, water, glass, design, satin, cot, stocking, grove, credit (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |                                                                                                                                                                                                                                                                                   | drawback, goodwill, thinker, largo, childe, font, culture, forme, delineation, watering, role, rote, sig, congratulation, cheque, dwelling, supporter, negro, furnishing, derby, fellowship, hock, sherry, sheen, teacher, pew, star, breadth, grace, seer, stock, haven, milliner, span, packet, gelding, viol, dais, roan, spa, warehouse, sharpness, anticipation, tweed, van, embryo, lodger, donor, bar, elocution, charm, sprinkling, diploma, equipment, connoisseur, loan, lane, adult, usefulness, vole, possessor, street, ruby, corpus, twin, remittance, lute, stationer, liege, giver, labourer, promotion, commendation, marquess, nook, pet, lad, music, report, card, zenith, china, champagne, saturday, hull, monday, rood, wednesday, longevity, vein, tuesday, thursday, grove, tavern, leader, yacht, service, lily, inn, admittance, receipt, season, birch, opus, jersey, class, ticket, eminence, renown, home, travel, coach (112)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | mill, cycle, holly, lamb, pendant, sunday, cross, dollar, baker, bio, infusion, sib, hook, rowing, fur, dining, art, yeoman, peacock, tine, willingness, rainbow, shoemaker, seller, sip, pavilion, ointment, loo, auto, trout, lens, focus, prism, bro, recreation, sphere, lace, steel, mademoiselle, arbour, churchman, corps, ling, park, gratification, expansion, faithfulness, tun, pearl, square, richness, sis, ax, fluency, nut, aurora, liter, stability, afternoon, sou, fervour, warren, maple, carpenter, tea, cot, proficiency, primrose, tin, belle, friday, space, altitude, plum, fitness, fir, fertility, weaver, punch, lime, board, owner, coal, cup, ilk, emerald, fulness, ger, rye, edification, universality, banker, ore, cam, jeweller, tad, dairy, butler, boy, fresco, fishing, ark, bedding, mountain, gel, field, pantheon, chocolate, beet, hawthorn, talent, canal, coming, quarter, radius, flower, chi, pic, colour, astronomer, oil, teen, evening, frock, tot, fleet, teaching, mechanism, rosemary, reservoir, stocking, aerial, filling, walker, vet, con, student, eon, sim, believer, penny, widower, elm, credit, artist, solidity, tan, continent, schooner, ivory, glass, quay, ride, contentment, basket, app, exposition, gardener, globe, muslin, author, deacon, success, cert, eld, excellence, summer, inspiration, knighthood, plat, poplar, lyric, lemon, tower, tip, enamel, cape, mount, porcelain, wood, composition, atmosphere, autumn, bow, compilation, tee, commemoration, distinction, brass, tobacco, spot, model, tic, diamond, acuteness, conception, village, wharf, chair, ale ... (369)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| +Pos+        | tact, organization, recognition, lack, missionary, training, lending, regeneration, consecration, incorporation, trait, critique, musician, command, career, value, advent, enthusiast, accord, consummation, learner, par, arc, follower, boon, conference, ordination (27)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | singer, lecturer, encore, communication, goo, goldsmith, chairman, chef, aid, forte, era, contribution, gentlewoman, niece, list, elder, germ, duo, builder, engineer, herald, harp, jubilee, intuition, solo, cathedral, bond, gilt, graduate, cleanliness, comp, banking, household, vocabulary, anthem, oratorio, guild, madame, painter, organist, beau, surveyor (42)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | manufacturer (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |                                                                                                                                                                                                                                                                                   | chef, watt, lecturer, entirety, trait, germ, graduate, goldsmith, aid, forte, lending, thrift, harp, guild, mater, fete, advent, award, accord, toilette, incorporation, consummation, herald, duo, boon, discoverer, glee, presentation, gem, baton, toilet, flora, carol, banking, repute, comp, rarity, gilt, doc, collation, jubilee, communication, tuition, builder, engineer, par, capital, cheapness, homo, stud, advertiser, housewife, unit, salutation, delivery, drawing, periodical, verity, prodigy, information, refreshment, guidance, circle, subscriber, exchange, stipend, beau, quota, villa, ballad, bust, certificate, maker, order, arc, term, footing, price (78)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | goo, learner, regeneration, atlas, cleanliness, follower, naturalist, deli, enthusiast, combination, store, gentlewoman, intuition, governess, illumination, chairman, value, freehold, elder, consecration, fount, evangelist, compass, observatory, staple, critique, command, hooker, painter, garland, company, farming, motto, niece, committee, vocabulary, embroidery, surveyor, mason, livelihood, manhood, maturity, repast, pastor, cathedral, mathematician, harvest, journal, fountain, abundance, vine, sustenance, ordination, fineness, synopsis, woodland, stature, microscope, elasticity, flavour, bride, mineral, octavo, chart, fragrance, complexion, radiance, brightness, perfume, verdure, lexicon, treasurer, nourishment, silver, age, volume, fruit, vegetable (78)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| +Qual        | scrutiny, solemnity (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | coming, lip, brandy, breadth, mood, noon, clock, star, chill, brick, ringing, zoo, contact, hurricane, flash (15)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | precision, hand, water, way, head, bucket, appearance, foot, conversation, countenance, glass (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | bouquet, railing, belt, sea, loft, piece, curve (7)                                                                                                                                                                                                                               | bouquet, tome, blackness, scrutiny, nabob, mood, drawing, breadth, star, ringing, vegetation, stranger, sort, corpse, solemnity, conversation, highland, lying, halt, way, circle, lily, beginning, term, horizon, piece, diversion, place, part (29)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | lip, bucket, holly, sea, atmosphere, foot, precision, chest, fawn, moon, curve, coming, casting, gust, chill, action, dragoon, turnip, space, side, citadel, heaving, flash, windmill, table, rigging, spot, appearance, fineness, brake, moment, countenance, glitter, brandy, summer, noon, likeness, draught, stone, belly, hurricane, night, rose, parapet, linen, resemblance, hand, sparkle, brick, horse, suspicion, earth, perception, glass, gate, bay, blanket, iron, face, breath, whiteness, shore, head, century, brass, camp, break, shot, weather, water, chimney, visage, morning, woman, silver, honey, wall, ground, anchor, river, battle, brightness, nature, door, soldier, heart, harbour, winter, breast, gold, length, bottom, sentiment, hour, tree, boat, sensibility (97)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| +Qual+       | impetuosity, coldness, irony (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | storm, comrade, snow, moor, melancholy (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | light, figure, shadow, helm, perspiration, scalp, mould (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | muff, moss, rill (3)                                                                                                                                                                                                                                                              | demon, rill, sheen, haze, greeting, lizard, pallor, allure, shadow, clutch, fern, perspiration, helm, lure, glen, handling, irony, moor, knight, texture (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | scalp, figure, leaf, bark, profile, lance, rainbow, comrade, lair, eve, cleft, flower, roughness, light, impetuosity, horn, dragon, hardness, mountain, twig, touch, snow, colour, daylight, track, emerald, expression, melancholy, scene, coldness, imagery, approach, storm, sun, beech, hair, humour, window, almond, colouring, smell, eye, serpent, wind, marble, frost, grass, imagination, sword, nose, rock, beauty (52)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| +Quant       | transportation, career, ruler, advent, incorporation, consummation, deduction, conflict, threat, confiscation, suppression, sneer, critique, privation, synod, disaster, licensing, outcry, multiplication, magnitude, correction, appropriation, goodwill, habit, default, bane, art, comment (28)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | lecturer, vita, encore, echo, singer, cycle, bishopric, sympathy, indigestion, diocese, partnership, herald, niece, rote, sig, quid, bass, relict, tuesday, feud, rite, clue, refutation, dollar, dissent, victor, reliance, intemperance, commissioner, anthem, anarchy, forte, county, disaffection, eon, feature, rashness, chef, earldom, summary, arbitration, panic, recitative, exemption, aid, plaintiff, disregard, argent, sink, lodger, executor, council, valuation, embryo, role, baker, slaughter, compensation, pang, pontiff, solo, promoter, limitation, auto (64)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | accommodation, sherry, boa, blessing (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | jib (1)                                                                                                                                                                                                                                                                           | advent, lecturer, rental, consummation, numbering, fuss, rote, herald, threat, ruler, flora, role, incorporation, sig, chef, domain, rite, feud, honor, clue, temp, privation, confiscation, mater, goodwill, jockey, victor, sherry, fare, disaster, partnership, sprinkling, dismay, forte, tsar, lodger, attendant, jib, deduction, prelude, licensing, recitative, embryo, sewer, suppression, aid, plea, extension, doc, banquet, papa, miser, arbitration, stipend, novelty, collateral, correction, periodical, trip, music, toast, harpsichord, joke, minority, naming, section, deposit, jam, vole, counting, incentive, sneer, card, publishing, concord, giver, guide, schedule, rebus, vita, congress, dissent, stationer, transaction, quid, reduction, promoter, guidance, nonsense, union, toil, attendance, crisis, tuesday, sway, renown (96)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | dollar, fusion, distribution, pop, indigestion, echo, seminary, indemnity, establishment, spec, multiplication, suicide, cycle, valuation, progeny, bishopric, sou, appropriation, fob, vid, tram, coke, auto, para, accumulation, default, donation, pit, peerage, barony, panic, refutation, dud, ruling, camera, mite, phi, freehold, fiction, operating, mamma, sos, magnitude, commissioner, partner, art, bane, bias, client, baker, pantheon, selling, limit, boa, enumeration, pang, result, litany, delusion, taxation, pastor, maze, guardian, pig, evangelist, gal, critique, niece, clemency, repast, flu, ritual, thankfulness, omission, contract, option, monopoly, dolly, predecessor, qualification, augmentation, principality, limitation, inventory, voting, reservoir, renewal, education, derivation, lam, disaffection, arithmetic, eon, reliance, comment, cardigan, connexion, expediency, diocese, summary, hen, imposture, relict, parent, lab, outcry, compilation, knighthood, surplus, intemperance, committee, offspring, virgin, purchaser, foresight, syntax, slaughter, curse, plaintiff, sinner, council, vow, rem, rashness, earldom, executor, triumph, sedition, octavo, exemption, bill, abs, number, pontiff, division, county, fate, forgiveness, journal, delay, dividend, benefactor, anarchy, woe, obstacle, disregard, outrage, argent, majority, imprudence, invitation, supplement, blessing, alum, compensation, jurisdiction, presumption, synod, folly, immortality, duchy, repentance, bondage, dictionary, contraction, freedom, eternity, redeemer, redemption, vegetable, borough, manuscript, claim, vengeance, ignorance, chronology, benevolence, guilt (178)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| +Quant+      | congratulation, case, endowment, absolution, organization, regeneration, veto, training, lunacy, inaccuracy, boon, archdeacon, consecration, ordination, philology, dealing, advert, abbe, layman, delegate, fellowship, exposition, extinction, lunatic, record, canon, formula, license, purgatory, rumour, chancellor (31)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | abstinence, list, derby, ego, era, remission, mania, reservation, duo, opus, monday, sunday, hierarchy, labor, viscount, banker, burglary, dean, concession, sine, oratorio, cheque, priesthood, renunciation, unwillingness, misconduct, divorce, inquest, axiom, friday, willingness, widower, justification, denial, complication, indictment, data, chairman, celibacy, exclusion, universality, deanery, implication, circuit, commune, choir, poultry, barrister, genesis, graduate, stability, robbery, infallibility, convocation, guild, organist, budget, turkey, epidemic, chronicle (60)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | fee, agency, chemistry, equity, machinery, salvation, concert (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |                                                                                                                                                                                                                                                                                   | veto, draft, cheque, mania, congratulation, stoppage, boon, labor, endowment, temperance, largo, legation, deficit, commune, ledger, tributary, directory, advert, assemblage, derby, scrip, recording, lunacy, sine, data, mining, duo, adult, holder, graduate, rating, award, guild, budget, notoriety, abstinence, stake, examiner, unit, umpire, racing, fellowship, enlargement, lunatic, antiquarian, digest, delegate, stud, confederation, donor, cation, turkey, extraction, asylum, codex, sequence, dating, employer, participation, criterion, persona, renunciation, tuition, presentation, opera, jus, formula, ego, promotion, disuse, speculation, chili, birthday, transit, passenger, teacher, divorce, tithe, couple, competitor, inauguration, possessor, pedigree, servitude, probation, usefulness, manual, messiah, school, opus, transport, advertiser, intro, abbe, executive, chronicle, survey, bookseller, task, monday, genus, vacation (102)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | regeneration, reviewer, remission, reservation, oversight, willingness, suffrage, governess, layman, inaccuracy, claimant, complication, concession, area, sis, formation, store, solving, packing, fart, unwillingness, koran, implication, observatory, librarian, annihilation, production, sunday, orphan, axiom, stability, banker, trespass, hierarchy, inquest, extinction, misconduct, sufficiency, interment, ordination, foreman, tutor, cession, license, universality, exclusion, bribe, birthright, consecration, dowry, absolution, unanimity, rumour, proficiency, fitness, solicitor, celibacy, dean, indictment, consultation, widower, stat, commemoration, attainment, chairman, philology, source, denial, epidemic, astronomer, chemistry, spelling, genesis, independence, synopsis, equity, recreation, teaching, munificence, priesthood, talent, livelihood, atonement, genealogy, algebra, bribery, rabbi, imposition, boiler, prosperity, viscount, edification, anniversary, barrister, synagogue, archdeacon, sustenance, canon, sanction, friday, acceptance, fallacy, liberality, drunkenness, eucharist, affluence, ton, embassy, deanery, advancement, deputation, petition, lexicon, purgatory, odium, convocation, adultery, rule, accession, infallibility, conversion, compendium, exposition, catechism, chancellor, commandment, laity, justification, food, validity, propagation, treasurer, decree, epitome, mathematics, conjecture, founder, fraud, sacrifice, baptism, acid, redress, principle, creation, salvation, excommunication, inference, famine, definition, essay, grandson, hypothesis (152)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| +Sing        | exaggeration, missionary, hostility, annoyance, temperament, confiscation, endowment, inaccuracy, conflict, allusion, exposition, musician, hearing, enthusiast, imbecility, rebuke, emotion, ruler, arrest, taint, license, disciple, arc, par, insurrection, modification, case, follower, sneer, piracy, record, fellowship, threat, appropriation, element, bankruptcy, ordination, interpreter, massacre, transfer, seizure, recoil, menace (43)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | singer, breadth, blight, intellect, axiom, tinge, oratorio, walker, explosion, goldsmith, movement, workhouse, rise, remonstrance, sink, mourning, cycle, blasphemy, chill, thrill, flash, concession, thinker, episode, victor, feature, agony, herald, believer, embryo, firing, coroner, tempest, panic, forte, ache, violence, anthem, dollar, chef, pang, divorce, indictment, misrepresentation, sine, region, infidel (47)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | manufacturer, sitting, succession, trail, visitor, protection, lawn, player, search, shaft, manufacture, fairy (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | cider, jade, manufactory (3)                                                                                                                                                                                                                                                      | annoyance, cider, hostility, endowment, trail, confiscation, imbecility, jade, bachelor, embryo, goldsmith, substitute, antiquarian, award, honor, chef, nursery, mystery, thinker, occupation, rarity, victor, presentation, confederation, guise, workman, herald, tailor, stab, vocation, thrill, fellowship, barrack, contingent, breadth, pretension, peculiarity, forte, suffering, threat, ruler, tribune, trench, sergeant, trip, stool, player, remittance, suggestion, passenger, par, pantomime, employer, sufferer, possessor, workhouse, transfer, menu, sine, ache, memorandum, remonstrance, manufactory, deposition, exclamation, notary, apparition, tiger, banquet, statuary, discoverer, terra, aide, host, equivalent, promotion, mourning, seizure, piracy, transition, taint, brute, wanderer, domain, attendant, speculation, ex, whisper, quota, wearer, fugitive, fairy, teacher, transport, stranger, salutation, divorce, fossil, commendation, transaction, peasant, hearing, sneer, refreshment, felon, navigation, performer, attendance, avail, leader, fiddle, genus, trifle, peace, victim, gesture, idiot, search, salute, arc, traveller (121)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | reviewer, blight, holly, encampment, naturalist, exaggeration, skeleton, razor, arrest, keel, tinge, follower, dollar, axiom, leech, sting, oversight, illumination, enthusiast, illusion, inaccuracy, scoundrel, shipwreck, license, opponent, piazza, chasm, belle, calendar, foam, border, fuel, concession, flash, store, turf, verge, suicide, appropriation, passport, chill, modification, rush, panic, sty, dash, suffrage, housekeeper, huntsman, avenue, recoil, predecessor, rebuke, tenure, fiction, para, cycle, combination, causeway, romance, believer, staple, viceroy, monster, murmur, ointment, explosion, extent, pang, pilgrimage, cradle, injunction, accomplishment, proprietor, episode, brake, impression, self, perch, magnet, taxation, shaft, astronomer, dialect, innkeeper, menace, emerald, robin, madman, walker, tract, rascal, scarf, pout, mouse, shock, employment, defiance, peep, jacket, driver, vase, consultation, lawn, pleading, fount, mortar, monopoly, risk, boundary, whirlwind, overflow, soup, ladder, personage, pavilion, production, interpreter, ceiling, region, connexion, bankruptcy, patronage, misrepresentation, partner, gallop, modem, turnip, delusion, practitioner, talent, trap, refinement, burial, goose, chest, exposition, amphitheatre, hammer, rudder, artist, coal, affliction, protection, errand, sanction, coroner, indictment, commotion, beggar, friar, pastor, grandfather, odour, acquisition, muslin, future, fibre, syrup, fever, basket, canoe, preparation, deputation, banner, structure, cancer, insurrection, cabin, recreation, babe, saddle, ability, massacre, ordination, mistake, friction, comedian, malady, gentleman, glitter, sheet, muscle, prose, disciple, blasphemy, smoke, slumber, speck, mansion, successor, frock, paste, infidel, reflection, ad, convenience, tension, stuff, ancestor ... (304)                                                                                                                                                                                                                                                                                                                        |
| +Sing+       | tact, organization, agitation, magnitude (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | warren, chairman, cloth, familiarity, termination, mount, beau (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | perfection (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |                                                                                                                                                                                                                                                                                   | moat, agitation, papa, beau, eminence (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | fob, distribution, glare, warren, chairman, removal, gratification, magnitude, alarm, breathing, caution, incapacity, familiarity, dressing, ascent, decay, cream, leather, presumption, humour, chalk, flour, adoration, scorn, contemplation, sovereignty, baptism, jurisdiction, consumption, mischief, mount, perfection (32)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| +Space       | solemnity (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | gateway, council, envoy (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | accommodation, affair, precision, kingdom, design, majesty, fringe, shaft, grit, throne, hall, modelling, concern, bed, stack, person, visitor, foot, fragment, king, bit, resolution, sitting, pail (24)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | mud, recess, loon, stall, kilt, filter, divan, belt, strip, hip, scroll, rill, curve (13)                                                                                                                                                                                         | filter, recess, strip, divan, grit, kilt, masonry, lass, greed, modelling, rill, pail, confederation, tweed, allure, ledger, equipment, suspension, affair, gateway, section, loop, media, bench, relation, solemnity, information, shelf, step, ottoman, fragment, minority, neutrality, beginning, peace (35)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | fen, shaft, specimen, skeleton, chasm, fringe, causeway, stack, ass, horseman, bit, kingdom, hoop, fir, court, curve, speck, side, foot, majesty, granite, space, sultan, folio, deviation, precision, speech, lava, elector, concern, slope, parapet, cardinal, chin, majority, scarf, trough, nation, congregation, purpose, nobility, woodland, city, boundary, utility, scroll, council, resolution, bed, century, scheme, rampart, assembly, cornice, thicket, throne, desire, promontory, excellency, declaration, apartment, province, king, camp, highness, parliament, consideration, membrane, person, sheriff, objection, coast (72)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| +Space+      |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | highway, forum, narrowness (3)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | material, spray, tank, soot, cairn, ledge, carpet, tent, ice, rim, dolphin, cage, pea, seam, rib, paw, rug, urn, tarn, cot, bulwark (21)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | peak, railing, slate, patch, jelly, screen, fence, frill, jib, moss, flue, disc, plaid, stile, loft, cone, sea, gear, kite, monkey, zinc (21)                                                                                                                                     | tank, flue, vantage, disc, frill, ledge, raft, jet, patch, cairn, jar, font, tool, tent, tributary, sewer, tuft, gorge, reef, haze, lizard, zinc, jib, inlet, pool, span, barrier, forum, gear, seam, glen, ice, paving, dolphin, rib, stair, clutch, stem, monkey, paw, car, stoop, communicating, dais, tarn, cone, gutter, shark, fore, nook, rug, dive, bulwark, labyrinth, mire, brink, bank, access (58)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | carpet, hilt, plaid, soot, chip, spray, sill, kite, rim, narrowness, pea, cot, urn, sea, portal, lobby, marsh, cage, tower, body (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| +Strong      | conflict, exposition, ballot, culture, endowment, incorporation, fervour, command, synod, case, training, theme, trait, delegate, formula, follower, mover (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | encore, singer, pint, opus, infusion, convulsion, street, goal, mall, fox, sig, labourer, nerve, finale, cleanliness, sympathy, engineer, stability, note, chairman, volunteer, advocate, promoter, intuition, labor, freeman, echo, bid, role, sharpness, forte, intervention, argent, consistency, beau, flash, biographer, hierarchy, region, baker, vindication, ear, recitative, harp, arbitration, acre, rider, embryo, painter, corps (50)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | precision, filly, sherry, fad, zeal, mirror, agency, proof, perseverance, plumage, grove, ram, series (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | jib, peak (2)                                                                                                                                                                                                                                                                     | role, jet, convulsion, baud, labor, culture, deficit, legation, lye, fare, jib, endowment, quadrille, sig, trait, toilette, theme, forme, incorporation, toast, charm, note, spark, temp, confederation, reef, circus, sherry, formula, presentation, ballot, glee, toilet, sharpness, car, fore, fan, equipment, jig, delegate, joke, forte, strait, handling, novelty, rating, criterion, mover, host, opera, intervention, tune, harp, racing, recitative, harlequin, repository, engineer, row, pageant, gelding, embryo, banquet, tug, spectator, gem, executive, flute, firm, foe, paint, volunteer, section, evacuation, extension, beau, promoter, cation, school, trump, race, opus, hail, stationer, street, leader, trot, incentive, arbitration, statuary, survey, grove, manual, fiddle, warehouse, strike, peal, information, limo, bust, apparatus, rebus, labourer, labour, character, salute, sage, exhibit, game, bard, wit (111)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | reviewer, cleanliness, hook, fox, follower, vert, infusion, file, puff, stability, opponent, pad, steadiness, freeman, mirror, observatory, advocate, combination, booth, razor, biographer, fir, intuition, rider, fork, enamel, legislator, cotton, demonstration, formation, fervour, pact, suffrage, sweep, recipe, production, huntsman, bid, whit, pen, bow, precision, baker, lock, flash, celerity, consistency, trout, recommendation, predecessor, fuel, aerial, ruling, region, foreman, plumage, beet, causeway, drama, physics, horn, liberality, vet, coal, repast, echo, accomplishment, pack, chairman, nerve, tan, ram, ear, instrument, exposition, spear, committee, command, attainment, chariot, warrior, preparation, tackle, painter, ordnance, touch, pin, shape, hue, tilt, admiration, flu, fitness, knot, witness, unity, perseverance, tin, company, beneficence, munificence, view, hierarchy, hammer, critic, flavour, countryman, defender, bribe, roar, plume, blaze, ladder, corps, ore, rudder, flow, ton, abundance, pan, qualification, knighthood, cane, flame, tincture, telescope, industry, mason, strain, printer, success, song, mu, eruption, breeze, zone, collection, dance, complexion, root, ski, torch, ability, pint, bee, palm, emblem, summit, tension, humour, schooner, ivory, comedy, servant, whip, understanding, arrow, acquaintance, dish, stature, argent, harness, poet, mix, admirer, glow, harm, accuracy, prowess, eye, age, wig, integrity, acre, tongue, pair, proverb, firmament, clearness, lyre, sex, synod, impartiality, musket, delicacy, dexterity, phrase, pistol, proof, zeal, sky, crop, delight, barometer, quart, thunder, charcoal, emulation, philosopher, virtue ... (207)                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| +Strong+     | tact, canning, organization, ruler, recognition, goodwill, regeneration, consecration, dealing, veto, philology, delineation, career, regime, advent, musician, fluency, champion, supporter, advert, retention, sphere, modification, missionary, sect, interpreter, value, art, ordination, churchman, element, reformer, learner, fellowship (34)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | lecturer, cycle, era, intellect, list, array, urge, aid, bass, focus, builder, graduate, feature, commune, thinker, derby, movement, chef, solo, link, partnership, concession, universality, comprehension, interpreting, guild, axiom, walker, fertility, data, jubilee, organist, forum (33)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | style, merit, machinery, player, neatness, manufacturer, elegance, species (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | pattern, rifle, disc, zinc (4)                                                                                                                                                                                                                                                    | canning, commune, goodwill, draft, veto, graduate, watt, lecturer, advert, baton, temperance, regime, chef, supporter, advent, guild, ruler, urge, zinc, aid, disc, jockey, retention, rally, umpire, player, discoverer, tributary, thinker, sect, competitor, stud, lever, prefect, delineation, mining, array, assemblage, operator, data, award, derby, aide, directory, repute, builder, media, coup, patentee, advertiser, challenge, periodical, display, partnership, antiquarian, donor, fellowship, music, digest, bookseller, forum, band, elocution, bar, applause, suggestion, unit, teacher, commendation, guide, pledge, heating, guidance, jubilee, transit, speed, task, aim, tuition, china, gallantry, performer, hull, maker, inventor, sway, subscriber, experiment, renown, citizen, avail, praise, printing (93)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | gauge, naturalist, atlas, regeneration, fusion, reformer, cycle, pendant, expansion, hooker, modification, coke, illumination, steel, learner, concession, pantheon, champion, bias, workmanship, universality, occult, interpreter, astronomer, cable, focus, consecration, churchman, talent, magnet, art, proficiency, student, neatness, camera, excellence, fluency, mechanism, indication, electricity, philology, axiom, politician, comprehension, trophy, style, motto, evangelist, ingenuity, compass, merit, mercury, prism, microscope, sphere, interpreting, value, richness, compilation, bayonet, practitioner, solidity, education, synopsis, foresight, agility, quickness, mathematician, fertility, weapon, stat, ordination, caution, statesman, discernment, inspiration, reputation, reservoir, globe, gun, nicety, composition, professor, taste, teaching, science, walker, boldness, species, triumph, patriotism, sobriety, symmetry, cylinder, conception, engine, penetration, author, elegance, invention, veracity, orator, probity, comet, wire, sagacity, brass, lexicon, journal, mineral, variety, diversity, pamphlet, erudition, scholar, honesty, metal, publication, shield, literature, frigate, skill, historian, armour, patriot, compendium, monument, exactness, interpretation, eloquence, language, genius, size, essay, epitome, tube, analogy, definition, treatise (139)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| +Time        | organization, irritation, temperament, irritability, cessation, allusion, supporter, sneer, value, tone, endowment, annoyance, enthusiast, detention, lifetime, relapse, habit, stagnation, strife, modification, follower, visitation, art (23)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | burst, explosion, brandy, cloth, blight, anticipation, echo, feeling, feud, nerve, peninsula, trash, tinge, longing, decease, mutton, rote, movement, pint, liver, reservation, levity, gladness, rapture, oratorio, chaplain, lamb, biographer, thrill, chill, flash, harp, valuation, inflammation, failure, apoplexy, retribution, ride, tea, bid, role, terrace, madame, embryo, pang, week, labor, beau, entry, firing, vent, rapidity (52)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | exertion, sherry, performance, filly, strength, constitution, concert, plumage, lustre (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | wreath (1)                                                                                                                                                                                                                                                                        | supporter, feud, stoppage, role, irritability, watering, bachelor, anticipation, longing, rote, lifetime, endowment, sherry, annoyance, rehearsal, nursery, cessation, flora, harp, retribution, rouge, labor, gathering, embryo, trash, dismay, mining, chime, thrill, champagne, birthday, transformation, inauguration, pageant, pilgrim, criterion, detention, sneer, disuse, opera, trump, fossil, zenith, china, messiah, paint, peninsula, certificate, vent, beau, whim, refreshment, attire, season, toil, haste, entry, renown, strife (59)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | blight, valuation, store, camera, liver, tinge, drug, scent, echo, follower, fresco, biographer, irritation, lamb, explosion, stagnation, enthusiast, baking, reservation, value, revival, governess, brood, dearth, mourner, festivity, interlude, cherry, apoplexy, flash, modification, specie, acidity, visitation, plum, nerve, renewal, chill, cup, bile, burst, diary, sou, baroness, bottle, bid, affliction, pang, temerity, enamel, art, composition, atmosphere, whirlwind, commemoration, fashion, glare, colour, gladness, excellence, suicide, avidity, property, pavilion, liquor, efficacy, growth, plumage, talent, brandy, relapse, pen, flower, wedding, cookery, existence, tenure, connexion, painting, syrup, rum, mixture, blush, goose, disquisition, chaplain, resistance, mixing, ginger, hair, brain, anniversary, ride, flavour, brigade, odour, alkali, kiss, sparkle, persuasion, glitter, venison, prospect, twilight, tincture, tea, colouring, decay, spleen, vibration, zephyr, smell, splendour, soap, independence, mix, honey, contemplation, flesh, dance, wreath, discord, perfume, offspring, diminution, pudding, week, rapidity, lustre, sugar, compliance, constitution, taste, pension, levity, fermentation, invention, blossom, pressure, complexion, shower, science, bloom, juice, gale, sulphur, wax, strength, elasticity, glow, delight, nourishment, mutton, boil, blaze, skin, sweetness, mu, fragrance, bud, gleam, oats, fruit, mind, food, decease, despair, quart, climate, hunger, evaporation, pint, rapture, inheritance, frost, brightness, pound, intercourse, acid, vivacity, breeze, prey, inflammation, wine, beef, moisture, stomach, gallon, dew, extravagance, firing, powder, vinegar, analysis, gold, barley, heat (197)                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| +Time+       | regime, career, advent, training, privation, critique, threat, absolution, agitation, theme, default, longevity, delirium, ascendant, canon, mover, heroine, weariness (18)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | encore, childe, lecturer, indigestion, era, herald, bishopric, finale, singer, episode, rite, debility, discouragement, goldsmith, intoxication, duo, cycle, mania, note, mood, unhappiness, termination, budget, partnership, punch, bitterness, derby, meal, reckoning, chef, meantime, relaxation, intemperance, closing, household, allowance, sine, infatuation, premium, victor (40)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | chemistry (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | spring (1)                                                                                                                                                                                                                                                                        | regime, childe, budget, legation, advent, lecturer, privation, mania, fete, meantime, herald, threat, debility, rite, carol, glee, chef, goldsmith, note, duo, sensation, theme, novelty, effusion, ascendant, recording, agitation, honor, temperance, discouragement, closing, greeting, racing, mood, elocution, mover, sine, digest, pantomime, directory, bidding, plight, prodigy, prelude, victor, derby, suggestion, dating, quadrille, omen, teacher, partnership, occupation, banquet, sport, reckoning, music, correspondence, joke, masque, toast, relaxation, stipend, asylum, survivor, periodical, attendant, longevity, pledge, generation, dissolution, troth, leader, retirement, adieu, heroine, luck (77)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | indigestion, seminary, dram, default, punch, drama, bishopric, episode, eve, drunkard, frolic, sales, cycle, critique, production, practitioner, ferment, scandal, lull, autumn, lyric, recipe, unhappiness, gaiety, demonstration, impression, aurora, preparation, mime, chemistry, intoxication, allowance, mamma, delirium, farce, farewell, recreation, comedian, canon, parting, malady, merriment, ascension, slumber, repast, observance, infatuation, cruise, muse, option, weariness, waking, bride, dissipation, triumph, dawn, absolution, digestion, intemperance, appetite, attack, adversity, bitterness, literature, midnight, dream, remembrance, maturity, nurse, bliss, spring, idleness, consumption, sleep, mirth, vengeance, remorse (77)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| +Vice        | irritation, threat, absence, allusion, disaster, semblance, accomplice, massacre, delirium, irony, warning, extinction (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | burglary, papacy, disregard, distrust, disapprobation, panic, refutation, slaughter, vehemence, antipathy, narrowness, infidel, reprobation, epidemic, scepticism, explosion, divorce, paradox, penitence, chill, unwillingness, tempest, explanation, impeachment, apoplexy, infirmity, wail, earthquake, termination, robbery, mob (31)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | security, occasion, inclination, safety, intention, thing, succession, concern, neighbour, throne, instance, posterity, mercy, notion, idea, monarch, reason, king, event, glory, equity, prejudice, preservation, protection, happiness, performance, empire, business, argument, magistrate, countenance, wife, zeal, shadow, soul, meaning, family, example, bed, constitution, soot, creature, answer, end, government, judge, frame, conversation, agency, rat, fragment, companion, providence, moth, elf, bulwark, cage, doctor, trouble, resolution, species (61)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | jelly, stile, belt, moss, slate, strip, bite, spider (8)                                                                                                                                                                                                                          | threat, barrier, strip, papacy, mystery, stab, wail, disaster, reverie, tool, puzzle, curb, stoppage, reprobation, plea, base, semblance, shadow, absence, blackness, speculation, gutter, magistrate, elf, rouge, protest, doctor, preservation, irony, sensation, system, felon, gesture, safety, administration, bulwark, divorce, juncture, companion, thinking, frame, adventurer, exit, occasion, fragment, warning, conversation, bourbon, antagonist, transition, riot, defending, memory, intent, state, transaction, dissolution, business, resort, crisis, separation, diversion, example, judge, correspondent, curiosity, refuge, making (68)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | howl, progeny, coalition, accumulation, gust, rack, suicide, opponent, froth, carnage, rhapsody, panic, narrowness, charge, caprice, disapprobation, kicking, rat, lament, removal, festivity, shipwreck, rush, merriment, ghost, parody, refutation, epidemic, maze, repetition, paradox, drowning, existence, soot, fable, vermin, accomplice, dirt, den, moth, concern, dearth, bite, irritation, detriment, protection, vigilance, infidel, contest, commonwealth, extinction, unwillingness, apoplexy, procedure, massacre, toad, equity, instance, indulgence, neighbour, antipathy, conclusion, countenance, circumstance, government, thing, chill, temptation, inability, exultation, adversary, pigeon, inclination, event, impudence, humanity, answer, obscurity, soliloquy, prejudice, whirlwind, security, deviation, distrust, posterity, detestation, unanimity, clemency, employment, spider, slaughter, impeachment, revolution, omission, delirium, disadvantage, frog, pardon, rag, timidity, scene, prevention, trap, rule, explosion, wife, artifice, consciousness, explanation, adulation, blush, idea, extent, passion, caution, robber, partiality, intention, rabble, fallacy, infirmity, diminution, deed, paganism, disregard, crying, politics, interposition, despotism, englishman, imitation, popery, fright, monarch, tendency, king, multiplicity, abdication, throne, glory, villain, dungeon, reparation, revenue, trading, inequality, ague, adversity, speech, remembrance, trade, legislature, scepticism, abhorrence, scruple, empire, restraint, redress, plague, solitude, behaviour, tumult, recourse, indifference, temper, end, notion, commotion, decay, astonishment, people, antidote, torrent, murder, felony, swine, presence, presumption, defect, realm, belief, creature, killing, pestilence, decency, contradiction, sacrifice, adherence, constitution, reason, envy, satire, thief, soul, bed, contempt, penitence, continuance, reign, apology ... (315)                                                                                                                                                                                                               |
| +Vice+       | deception, desertion, sacrilege, privation, annoyance, coldness, distraction, loathing, strife, inaccuracy, penalty, imbecility, confiscation, peril, mortification, recoil, relapse, wretchedness, sneer, weariness, menace, stagnation, disorder, carelessness, contagion, interruption, blunder, detention, intrigue, taint, offender, chagrin, outcry, insurrection (34)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | retaliation, perversion, inconsistency, discouragement, trash, blasphemy, bitterness, remonstrance, feud, rudeness, fanaticism, intoxication, disaffection, evasion, rashness, levity, banishment, revolt, misrepresentation, anarchy, mania, failure, blindness, intemperance, unhappiness, mockery, infatuation, bloodshed, perplexity, prohibition, impertinence, retribution, pang, blight, meddling, baseness, hardship, sob, agony, humiliation, misconduct, schism, rash, complication, melancholy, atheism, sink, violence (48)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | blast, consequence, affair, reptile, injury, wretch, perspiration, herd, snail, cur, beetle, trail, dart (13)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | mud, railing, muff, fence, mule, rake, patch, crocodile, jade, monkey (10)                                                                                                                                                                                                        | demon, mule, mania, feud, sob, privation, annoyance, reptile, rake, desertion, imbecility, retribution, loathing, incredulity, humiliation, trail, spy, freak, plight, discouragement, trash, suffering, sham, yawn, lust, poisoning, confiscation, peril, sufferer, dismay, pallor, remonstrance, jade, monkey, exclamation, brute, perspiration, mist, vagrant, patch, disuse, brand, apparition, nuisance, stain, clutch, intrigue, inoculation, chagrin, miser, vagabond, rout, servitude, consequence, prevalence, medley, mire, herd, chatter, affair, crow, dilemma, detention, gaming, nonsense, filth, waste, sneer, strike, idiot, snare, taint, victim, rescue, cell, whisper, joke, quack, hazard, strife, haste (81)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | deception, retaliation, detection, recoil, fanatic, fatality, annihilation, contention, irregularity, scoundrel, puff, dupe, bloodshed, grin, drunkard, bully, menace, blast, malignity, meddling, stagnation, glare, sting, sacrilege, carelessness, evasion, temerity, impropriety, hush, rust, hiss, blight, puppy, delusion, monster, complication, distraction, impertinence, pang, disgust, perversion, jest, crocodile, inaccuracy, scourge, madman, gasp, propensity, scandal, rascal, soldiery, misrepresentation, silence, drug, snail, lamentation, swearing, prohibition, snake, mask, indignity, barbarism, knave, invader, venom, unhappiness, destruction, blunder, mortification, pique, trick, catastrophe, insensibility, fanaticism, beetle, beggar, moment, moan, disaffection, blindness, sloth, intoxication, assassination, laughter, bubble, murmur, witchcraft, misconduct, rudeness, swarm, brood, indiscretion, profligacy, slander, groan, infatuation, mockery, jargon, brutality, derision, weariness, revolt, relapse, hypocrite, imposition, coward, uncertainty, hardship, complaint, frenzy, blasphemy, alarm, disturbance, rap, dart, grievance, devastation, spleen, frown, reproach, inconsistency, penalty, disorder, malice, confinement, baseness, malady, ferocity, deceit, heap, wretch, owl, licentiousness, desolation, chaos, rage, bitterness, wretchedness, mistake, dissipation, infidelity, affront, offender, iniquity, deformity, curse, fool, negligence, depravity, imposture, pretext, bribery, contagion, libel, coldness, discord, rashness, intemperance, distress, load, invective, vexation, expulsion, imprudence, odium, doom, falsehood, melancholy, interruption, disgrace, rash, disguise, risk, conflagration, sedition, meanness, banishment, superstition, impunity, traitor, terror, injury, fate, scorn, darkness, obstinacy, faction, shame, heretic, amazement, combat, forgery, idleness, torment, drunkenness, crush, insolence, levity, perplexity, hell ... (339)                                                                                                                                                                                      |
| +Virtue      | delineation, drawback, goodwill, critique, culture, exposition, fluency, command, canning, amount, sphere, grace, fervour, art, advert, fellowship, arc (17)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | cloth, cycle, tuesday, sunday, lane, monday, chant, wednesday, friday, mount, intellect, pint, street, thursday, cross, coffee, league, labourer, derby, childe, goldsmith, park, horseback, fortnight, tea, faithfulness, coming, width, strand, sig, haven, manufacturing, oil, infusion, afternoon, walker, october, baker, gentlewoman, saturday, warren, meal, butler, eon, peninsula, mile, parcel, brandy, app, poultry, link, contentment, beach, opus, sharpness, rote, dwelling, september, earnestness, commandant, evening, gateway, mall, lace, fertility, quarter, tile, cam, lamb, terrace, argent, mutton, depth, ride, clock, willingness, priory, january, acre, gallery, ire, universality, noon, banking, expiration, dollar, candle, cheque, stability, square, role, county, ling, chi, castle, acuteness, snow, liter, deacon, channel, grocer, suit, brigadier, ward, wheat, gazette, penny, coat, mademoiselle, morn, zoo (111)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | sherry, water, material, biscuit, design, glass, cot, satin, pail, oar (10)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | manufactory (1)                                                                                                                                                                                                                                                                   | goodwill, font, drawback, forme, childe, role, cheque, culture, sig, delineation, goldsmith, hock, derby, span, advert, dais, pail, furnishing, sheen, tome, rote, watering, sharpness, roan, canning, packet, gelding, sherry, haven, fellowship, teacher, viol, baud, pew, manufactory, bar, spa, tweed, dwelling, pith, warehouse, car, banking, tuition, fife, van, grace, media, gateway, lever, tilth, remittance, milliner, homo, row, vole, corpus, usefulness, sprinkling, card, lane, hull, nook, street, loan, glen, burg, ruby, stationer, cornet, commendation, lad, elocution, promotion, champagne, ream, guidance, charm, possessor, china, labourer, peninsula, steeple, warming, music, report, monday, saturday, wednesday, congress, bookseller, birch, tuesday, statuary, thursday, opus, hoard, ballad, lily, gazette, receipt, inn, jersey, salute, writ, swimming, service, renown, arc, eminence, chaise, home, order, coach (114)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | mill, hook, holly, cycle, rowing, sib, dollar, fur, sunday, yeoman, baker, coo, keel, sailing, bio, cross, willingness, dining, art, peacock, infusion, lamb, prism, trout, oar, steel, rainbow, ling, loo, bro, fir, gentlewoman, lens, sip, stability, ointment, carpenter, pavilion, warren, maple, mechanism, ark, nut, cotton, faithfulness, tot, tin, kettle, richness, cot, ax, sou, lace, sis, tun, ilk, seller, coal, space, universality, critique, proficiency, mare, pearl, liter, tea, porch, afternoon, park, butler, ordnance, fluency, cam, friday, fulness, gel, lime, ger, quay, ore, effigy, tad, rye, square, solidity, primrose, beet, fitness, board, command, aurora, mountain, weaver, pantheon, dairy, canal, plum, jeweller, walker, fishing, quarter, fervour, sphere, chocolate, fertility, teen, hawthorn, cup, freight, bedding, schooner, eon, radius, owner, tackle, mademoiselle, sos, elm, reservoir, vet, eld, tan, tower, teaching, oil, plat, fleet, armament, bow, recreation, chi, wand, sim, coming, aerial, talent, consultation, putt, poplar, student, deacon, mercury, filling, edification, tip, dial, ire, booty, success, tilt, boy, biscuit, exposition, beach, sod, mead, cruise, basket, ton, ridge, tee, ride, rosemary, wharf, enamel, cutter, knighthood, muslin, snow, evening, app, causeway, tier, cert, ivory, inspiration, workmanship, ferry, flu, spire, colour, ensign, cape, bom, wad, bra, mount, beech, compilation, oat, brush, gun, lab, acuteness, lemon, ascent, flower, tic, cop, coachman ... (390)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Virtue+     | tact, recognition, organization, missionary, lack, training, incorporation, consecration, lending, regeneration, musician, advent, value, accord, trait, consummation, learner, par, career, churchman, follower, ordination (22)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | encore, singer, lecturer, goal, breadth, goo, chairman, thinker, era, list, chef, forte, contribution, circuit, aid, germ, communication, star, duo, builder, focus, marquess, bass, solo, harp, jubilee, engineer, herald, intuition, cathedral, elder, anthem, graduate, cleanliness, bond, corps, guild, comp, gilt, oratorio, niece, vocabulary, surveyor, organist, data, painter (46)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | manufacturer, grove (2)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | bouquet (1)                                                                                                                                                                                                                                                                       | chef, watt, lecturer, germ, entirety, thinker, largo, baton, graduate, forte, guild, mater, aid, trait, advent, harp, incorporation, lending, thrift, award, duo, accord, bouquet, consummation, toilette, discoverer, presentation, doc, herald, fete, equipment, collation, gem, star, staff, toilet, repute, stock, breadth, stud, glee, jubilee, carol, builder, par, comp, diploma, advertiser, flora, cheapness, unit, data, gilt, engineer, delivery, liege, capital, rarity, communication, periodical, marquess, bible, information, salutation, rood, drawing, zenith, lute, stipend, refreshment, quota, subscriber, bust, leader, villa, certificate, grove, yacht, circle, engraver, footing, maker, price (83)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | goo, learner, atlas, regeneration, cleanliness, naturalist, pendant, deli, follower, combination, tine, illumination, intuition, chairman, store, expansion, churchman, freehold, hooker, compass, focus, evangelist, value, consecration, fount, corps, altitude, observatory, emerald, field, surveyor, astronomer, motto, company, elder, ascension, mason, embroidery, garland, committee, manhood, painter, telescope, vocabulary, porcelain, maturity, microscope, synopsis, pastor, vine, mathematician, elasticity, repast, ordination, palm, octavo, globe, cathedral, fountain, harvest, chart, stature, journal, abundance, fineness, sustenance, myrtle, firmament, woodland, niece, lexicon, warden, meridian, size, brightness, flavour, mineral, fragrance, professor, radiance, tabernacle, virgin, quarto, verdure, college, silver, perfume, treasurer, nourishment, plate, complexion, volume, latitude, orb, sum, fruit (96)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| +Weak        | sadness, penalty, martyrdom, loathing, exaggeration, drawback, distraction, weariness, interruption, solemnity, delirium, detention, wretchedness, chancellor, default, peril, visitation, offender (18)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | countess, exchequer, dwelling, ache, ending, intoxication, melancholy, remonstrance, chill, scaffold, sob, decease, failure, prohibition, blight, gaol, agony, castle, inconsistency, wail, priory, entry, convent, household, abbey, pang, clerk, dowager, exemption, disapprobation, afternoon, cheque, bloodshed, retribution, duke, earl, noon (37)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | affair, family, happiness, bed, foot, throne, consequence, security, interest, prejudice, mercy, visitor, carpet, king, wretch, treasure, resolution, crown, relief, concert (20)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | bouquet, railing, slate, sea (4)                                                                                                                                                                                                                                                  | sob, bouquet, loathing, wail, greed, drawback, ache, cheque, dwelling, nabob, desert, rouge, barrier, retribution, mystery, partition, troop, remonstrance, cologne, electorate, mister, plight, vacancy, deficiency, yawn, nuisance, posture, detention, peril, vagrant, scaffold, affair, solemnity, wardrobe, consequence, ending, beginning, treasure, drawing, entry, situation, restitution, memory, corner, state, intent, peace, monsieur, separation, victim, diet, content, finding, refuge, taking, descent, making (57)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | martyrdom, carpet, exaggeration, default, blight, chill, sadness, suicide, plantation, lamentation, drowning, dirt, penalty, foot, distraction, hush, prohibition, sting, residue, indignity, regency, melancholy, bloodshed, pang, drunkard, moan, weariness, madman, rent, intoxication, soliloquy, beggar, delirium, court, indulgence, sea, sloth, disapprobation, visitation, garret, impropriety, inability, profile, interruption, hardness, swarm, dormer, consciousness, alarm, vermin, residence, solitude, house, rupture, prejudice, propensity, landing, dungeon, chamber, groan, relief, blood, licentiousness, crying, gaol, wretchedness, popery, remainder, chancellor, baggage, custom, child, pretence, twilight, side, slumber, possession, bed, displeasure, day, speech, countess, afternoon, disturbance, castle, island, dissipation, keeper, offender, omission, pain, realm, march, fright, family, shelter, dignity, wretch, clerk, contrary, affection, revenue, shame, presence, weeping, languor, superstition, disobedience, pretext, odium, abbey, rebellion, importation, behaviour, childhood, insolence, burial, communion, suspicion, grievance, province, security, delay, resolution, oblivion, sickness, throne, payment, grandfather, abhorrence, insult, king, madness, petition, inconsistency, affront, thrust, land, refusal, breach, defeat, anger, departure, turning, ferocity, despair, time, vexation, indifference, confession, imprisonment, dust, tenderness, widow, immorality, exemption, priory, noon, happiness, privilege, horror, equator, prince, money, decency, inconvenience, dowager, prayer, princess, sinner, bondage, grass, agony, resentment, danger, gloom, pension, meanness, sleep, contempt, jurisdiction, hospital, decease, woe, convent, consent, apartment, offence, crime, scarcity, arrival, barbarity, torment, eternity, quarrel, necessity, hunger, remorse, brother, prisoner ... (230)                                                                                                                                                                                                                                                         |
| +Weak+       | annoyance, desertion, privation, coldness, irritation, irritability, carelessness, mortification, imbecility (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | workhouse, longing, bitterness, coming, banishment, hardship, unhappiness, mourning, trash, perplexity, childe, mockery, blasphemy, humiliation, gladness, misconduct, apoplexy, levity, sink, inflammation, penitence (21)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | wife, shadow, sitting, perspiration, darling, doctor, father (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | jelly, moss, room (3)                                                                                                                                                                                                                                                             | annoyance, privation, demon, desertion, reverie, humiliation, longing, wanderer, childe, pallor, suffering, shadow, darling, clutch, irritability, blackness, doctor, perspiration, trance, imbecility, mermaid, sufferer, apparition, trash, workhouse, pool, lizard, churchyard, bachelor, incredulity, dismay, room, nursery, mourning, chatter, gutter, stain, vagabond, corpse, filth, felon, thinking, nursing, lying, burden, idiot, brink, home (48)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | carelessness, fatality, shipwreck, unhappiness, dupe, apoplexy, hardship, verge, ghost, misconduct, mortification, solace, disgust, irritation, heap, sexton, gasp, wife, insensibility, mockery, complaint, gladness, finery, scandal, coldness, uncertainty, reproach, obscurity, coming, silence, desolation, ague, depravity, distress, infancy, banishment, prison, consternation, bitterness, confinement, perplexity, darkness, summer, loss, malice, drought, misfortune, slander, disgrace, derision, infection, idleness, night, fatigue, disappointment, terror, blasphemy, straw, rubbish, deformity, misery, negligence, neglect, rheumatism, timidity, brutality, floor, penitent, imprudence, amazement, pestilence, infamy, poverty, dread, consolation, calamity, grandmother, funeral, abyss, plague, morning, door, ingratitude, parlour, heart, penitence, weakness, indolence, levity, impatience, window, father, hanging, affectation, winter, compassion, exile, uneasiness, confusion, avarice, wickedness, aunt, hour, famine, death, pillow, inflammation, coffin, ignorance, guilt, joy (111)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| +Woman       | drawback, import, par, regeneration, suppression, lending, exposition, appropriation, modification, trial, arc (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | childe, faithfulness, perversion, explanation, tuesday, wednesday, convocation, madame, creed, width, council, vindication, germ, thursday, demolition, liter, exclusion, prohibition, mademoiselle, preface, divorce, gladness, schism, impeachment, infallibility, genesis (26)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | precision, article, hall, material, performance, promise, accommodation, design, intention, proof, kingdom, agency, commerce, preservation, foundation, ion, interest, prejudice, notion, argument, crown, discovery, sitting, favour, manufacture, inspection, idea, answer, power, antiquity (30)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | stile, doll, curve (3)                                                                                                                                                                                                                                                            | drawback, childe, germ, bible, cologne, import, par, watering, masonry, doll, lending, proximity, suppression, electorate, presentation, annex, partition, spa, demolition, terra, opening, nursing, preservation, mosaic, margin, cheapness, usefulness, divorce, tonnage, ottoman, restitution, intro, section, press, lily, equivalent, examination, information, chancel, veil, price, exhibit, neutrality, wednesday, peace, state, tuesday, order, giving, thursday, place, part, arc (53)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | coo, prop, article, regeneration, solving, creed, appropriation, supremacy, identity, sanctuary, promise, mademoiselle, regency, liter, ion, staple, expediency, expansion, constituent, precision, conformity, production, kingdom, preference, con, explanation, pearl, faithfulness, perversion, multiplicity, modification, prohibition, exclusion, abbess, removal, curve, renewal, abridgment, fiction, restoration, nun, impropriety, existence, finery, freight, utility, aurora, rosemary, preamble, litany, revival, conclusion, introduction, court, reference, morality, answer, sanction, thankfulness, efficacy, prejudice, impeachment, convocation, sequel, cession, progress, edification, gladness, matter, holiness, council, muslin, embroidery, tier, manufacture, confirmation, foundation, continuation, plan, sculpture, discovery, divinity, observance, journal, folio, intention, omission, arithmetic, propriety, appeal, catechism, coral, geography, solution, certainty, commerce, future, dispensation, propagation, exodus, genesis, octavo, collection, exposition, authenticity, frontispiece, work, purpose, treaty, variable, dedication, validity, popery, reformation, paganism, liturgy, act, province, nit, scheme, importance, jurisdiction, substance, supplement, proof, church, worship, trial, uncertainty, subject, decision, form, favour, conjecture, elasticity, empress, index, alliance, eucharist, origin, infallibility, cornice, faith, confession, bud, publication, reading, contrary, evidence, tie, superstition, objection, antiquity, clause, evaporation, alteration, twine, mythology, paragraph, schism, idea, consideration, notion, scripture, june, petition, density, violation, diversity, brightness, ordinance, power, tranquillity, declaration, statute, proposition, interpretation, saviour, quarto, volume, belief, importation, polity, sentiment, recourse, redemption, prophecy, religion, tapestry, sweetness, diction, ad, necessity, version, preface, extract, affection, history, charter, revelation ... (233)                                                                                                                               |
| +Woman+      | grace (1)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | household, rite, terrace, duo, convent, unhappiness, sob (7)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | happiness, satin, majesty, carpet, goodness, hive, concert, elegance (8)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | bouquet, tress, recess (3)                                                                                                                                                                                                                                                        | bouquet, tress, font, nursery, mystery, sob, sheen, recess, pallor, rouge, rite, knitting, duo, grace, gem, pet, drawing, separation (18)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | carpet, belle, mite, flower, lid, rainbow, growth, balcony, porch, overflow, ritual, lilt, communion, washing, vase, refinement, mum, unhappiness, plat, majesty, poppy, attachment, devotion, weaving, purity, fountain, richness, linen, happiness, prayer, baptism, syrup, goodness, amber, gentleness, convent, elegance, adoration, altar, nature (40)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| -Ambig       | exaggeration, congratulation, musician, learner (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | anthem, gateway, antipathy, sink, harp, willingness (6)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |                                                                                                                                                                                                                                                                                   | congratulation, knitting, harp, dismay, gateway, asylum, pantomime, eminence, travel (9)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | exaggeration, learner, bully, naturalist, willingness, hospitality, prism, antipathy, muslin, beech, rectitude, cheek, pudding, ferocity, deck, handkerchief, glow, fortitude, gloom, cream, anatomy, constancy (22)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| -Ambig+      | lunatic, archdeacon, archbishop, chagrin (4)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | mutton, singer, lace, indigestion, moralist, widower, liver, vocabulary, tempest, ringing, parentage (11)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | helm, filly, sherry, lustre, darling (5)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | screen, peak, recess, rifle, vest (5)                                                                                                                                                                                                                                             | lunatic, foal, helm, prelude, recess, vest, moralist, inlet, repute, mist, sherry, darling, examiner, ivy, quadrille, chagrin, harpsichord, rehearsal, parentage, miser, ringing, toast, relation, pedigree, militia, refreshment, supper, term, chaise, sway, coach (31)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | leaf, contention, sailing, lace, liver, indigestion, trout, gust, gin, noun, rosemary, dialect, speck, suicide, daylight, worsted, appellation, table, madman, manhood, mamma, mast, vocabulary, widower, matron, cup, cancer, shade, heroism, lustre, knot, pearl, diamond, mate, voting, chalk, lemon, poplar, multitude, mutton, disquisition, syntax, coward, archdeacon, velvet, infancy, pine, archbishop, duchess, midnight, cough, adultery, acid, tempest, laurel, waistcoat, testimony, age, yoke, shoulder, beef, anchor, sister (63)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| -Changed     | deception, regeneration, exaggeration, appropriation, martyrdom, layman, stagnation, arrest, checking, hearer, distraction, enthusiast, rebuke, mediocrity, inaccuracy, follower, reformer, irritation, churchman, sadness, accomplice, recoil, carelessness, extinction, sphere, fervour, proclamation, judaism, bane, learner, multiplication, sacrilege, default, penalty, courtesy, abatement, delirium, verdict, coldness, prosecutor, consecration, value, mortification, critique, formality, license, magnitude, menace, relapse, impetuosity, art, interpreter, courtship, archdeacon, champion, modification, visitation, rumour, absolution, fluency, bankruptcy, hindrance, massacre, blunder, impediment, philology, lord, ordination, disorder, licence, canon, stress, comment, disciple, command, calculation, weariness, exposition, confessor, reluctance, outcry, emotion, interruption, trial, synod, offender, contagion, chancellor, purgatory, wretchedness, insurrection, archbishop, martyr (93) | retaliation, goo, blight, lip, reservation, indigestion, liar, cycle, democracy, valentine, liver, adjournment, lamb, cross, tinge, tariff, bishopric, axiom, sound, infusion, freeman, unwillingness, sunday, shoemaker, fox, elder, ado, refutation, exaltation, lace, acquittal, park, echo, gentlewoman, celibacy, liter, prologue, intuition, perversion, lint, tenant, deafness, banker, stability, rider, auto, exclusion, prediction, remission, panic, dollar, episode, comrade, cleanliness, bearer, evasion, dit, gall, bloodshed, commissioner, chairman, implication, warren, baker, explosion, meddling, disaffection, vestry, gaol, allowance, inexperience, valuation, chaplain, concession, bombardment, paradox, complication, consistency, butler, inferiority, distrust, narrowness, faithfulness, willingness, rudeness, creed, denial, acquiescence, prohibition, inquest, biographer, ebb, burst, contentment, interpreting, ire, punch, coroner, recorder, ling, indictment, condemnation, diocese, chill, universality, argyle, flash, focus, revolt, disapprobation, coming, tide, impertinence, unhappiness, viscount, hardship, explanation, advocate, scepticism, cathedral, square, fanaticism, coquette, blindness, assent, melancholy, nerve, comprehension, barrister, niece, believer, apoplexy, mademoiselle, friday, misrepresentation, bid, widower, surveyor, app, limitation, walker, brandy, rector, vocabulary, snow, hierarchy, bacon, pang, intoxication, ban, dean, blasphemy, canto, theory, painter, gladness, priesthood, bologna, antipathy, tea, relict, epidemic, region, phraseology, nightingale, impeachment, misconduct, frankness, infirmary, bitterness, afternoon, vault, inconsistency, vicar, reliance, kinsman, eon, fertility, broker, hurricane, deanery, cam, plaintiff, brevity, banishment, corps, penny, storm, calmness, countess, oil, brick, depth, gentile, chi, executioner, deacon, apothecary, quarter, slaughter ... (295) | carpet, herring, cob, article, slit, pea, promise, soot, flight, blast, scalp, mirror, cove, ion, lath, shaft, oar, mat, spray, rim, dress, kingdom, figure, onion, snail, bucket, mug, rat, diadem, gad, biscuit, ornament, neatness, precision, bit, fringe, stack, majesty, wing, beetle, posterity, appearance, peg, neighbour, moth, countenance, merit, instance, cot, concern, protection, carving, discovery, satisfaction, thing, inclination, fishery, light, boa, stocking, security, party, lawn, glove, intention, style, vat, wife, perseverance, glory, cushion, credit, event, relief, world, effect, equity, foot, chemistry, empire, dart, comb, toe, urn, broom, commerce, pleasure, poker, bowl, plumage, government, paper, bed, constitution, answer, end, injury, assistance, prejudice, goodness, honour, copy, fluid, stalk, advantage, ram, animal, notion, foundation, wisdom, support, happiness, king, idea, family, cage, hand, throne, heaven, wretch, lustre, species, person, glass, study, capacity, grease, monarch, manufacture, head, proof, title, father, reason, mercy, perfection, distance, favour, soul, antiquity, resolution, strength, friend, motion, water, blessing, creature, authority, trouble, power, esteem, zeal, elegance, argument, salvation, fire, name, courage, providence, meaning (160) | plaid, puss, kite, rind, nymph, curve, lump, equipage, bite, crocodile, sea, spider, kit, morsel, wreath, scroll, spring (17)                                                                                                                                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| -Freq        | inaccuracy, lunacy, mover, abatement, incorporation, proclamation, delegate, par, arc, abbe, chagrin, learner (12)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | childe, inexperience, acquittal, burglary, ado, adjournment, infatuation, council, vita, aid, brigadier, recorder, rote, commandant, promoter, gentlewoman, evasion, ire, implication, prohibition, earldom, chi, zoo, executor, apothecary, duke, demolition, auto, bid, cam, broker, apoplexy (32)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | fee, fad, kingdom, king, ion, concern, hive, mutt, design, affair, grit, equity, security, cur, cairn, majesty, prejudice, cot, bucket, snail, mat, fort, mote, elf, crown, tarn, wick, gad, scalp, onion, foot, oar, bulwark, rib, answer, motion (36)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | muff, belt, railing, kilt, stile, haft, stall, punt, chine, rake, equipage, frill, coon, loft, tress, loon, jade, rill, runner, rift, cone (21)                                                                                                                                   | childe, asset, whore, heft, raft, tome, rime, contra, rill, punt, kilt, font, allure, thud, tress, mover, grit, rake, legation, frill, jade, runner, inlet, chine, rating, dispatch, nabob, rift, lunacy, forme, ration, gene, lye, rote, fife, evacuation, twill, hiring, playhouse, monitor, jet, fore, vantage, aid, fop, film, cairn, itch, cation, prance, tsar, elf, lure, publican, scan, incorporation, batter, electorate, deficit, delegate, umpire, wick, fan, lotion, trump, exam, annex, par, chagrin, persona, posture, larder, tarn, cone, mote, housewife, liege, plea, promoter, remittance, disuse, rib, ex, survivor, demolition, gull, milliner, vagrant, hatch, reward, gent, burg, masque, bulwark, diversion, elk, affair, pant, dive, restitution, felon, neutrality, foundling, halt, quota, repair, abbe, peace, vita, arc, order, wit (112)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | cult, ting, fen, tick, chip, liner, char, fice, prom, coo, bout, froth, caliph, prick, rhapsody, mallet, trill, whit, ion, muck, carnage, pore, ranger, onion, fatality, pate, auto, oar, bucket, progeny, ado, rant, pact, gradation, fart, ref, inaccuracy, steadiness, temerity, ger, scar, pap, celerity, interlude, guile, equipage, bus, subaltern, scalp, puppy, maze, birthright, kingdom, tike, putt, phi, vert, solving, learner, fir, acquittal, fem, epilogue, millet, con, rho, shopkeeper, pique, gnat, pad, vid, pout, cleft, dice, implication, inexperience, pal, pear, snail, gel, cot, brewing, oke, gunner, cel, multiplicity, gad, thrush, ping, gland, adjournment, tinsel, proclamation, parody, alloy, petitioner, deli, bully, windmill, turnip, mat, kin, ax, gal, concern, cad, teen, gentlewoman, evasion, preamble, mercury, sip, lam, regency, rig, roper, armament, prohibition, detriment, courtier, hoop, bro, abatement, infatuation, equity, sot, smuggling, rudder, beneficence, dryness, brine, brigadier, chi, dud, knave, dauphin, indemnity, fleece, towel, jeweller, ace, fineness, ire, gi, pom, fib, batt, crab, amity, sherlock, apoplexy, bid, king, foot, bib, ox, barge, digression, exuberance, ore, affront, confederacy, council, mediation, bail, caw, prince, prejudice, allegory, cancer, contraband, abridgment, rosemary, tab, rem, equation, pe, cardigan, ferment, bra, cam, pretence, jargon, burnet, fleet, court, bot, tanner, recorder, cop, rectitude, cure, broker, majesty, dad, spleen, ski, mint, brocade, invective ... (282)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |